## Data/MC Comparison for Brem-Induced Clusters

#### Comparison of Clusters in Data vs MC. This is to validate the signal reconstruction process. We compare cluster from Z->MuMu events in Data vs those from a DY->ZMuMu (50-120 GeV for MLL). Data is from 2023B&C, and MC is from the preBPix, normalized to the appropriate value 

In [1]:
import numpy as np
import pandas as pd
import uproot
import matplotlib.pyplot as plt
import sys
sys.path.insert(0,"../")
import mplhep as hep
import pickle
import glob
import ROOT as rt
import coffea
import awkward as ak
from coffea import hist, processor
from coffea.nanoevents.methods import candidate
from coffea.nanoevents.methods import vector
import os
import json

Warning in <TInterpreter::ReadRootmapFile>: class  HepMC::FourVector found in libSimDataFormatsGeneratorProducts.so  is already in libHepMC3rootIO.so 


Welcome to JupyROOT 6.28/00


/cvmfs/sft.cern.ch/lcg/views/LCG_103/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/cvmfs/sft.cern.ch/lcg/views/LCG_103/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/cvmfs/sft.cern.ch/lcg/views/LCG_103/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/cvmfs/sft.cern.ch/lcg/views/LCG_103/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  r

In [2]:
#import helper modules for muon scale factor computation
sys.path.append("/storage/af/user/aalbert/CMSSW_14_1_0_pre4/src/RazorCommon_correctionslib/RazorCommon/Tools/bin")
import importlib
import getMuonScaleFactor

#### Load ntuples as awkward arrays

In [3]:
ak.behavior.update(candidate.behavior)

def getLZDF(f,nEvents=-1,version="new"): #lazy dataframe with events that have cluster matched to probe muon
    events_raw = uproot.open(f)['MuonSystem']
    df = coffea.processor.LazyDataFrame(events_raw,entrystop=nEvents)
    start,stop = df._branchargs['entry_start'],df._branchargs['entry_stop']
    events = uproot.lazy(df._tree)
    #events = events[start:stop]
    return events

In [4]:
#paths
MC_paths = {"2022":"/storage/af/user/aalbert/CMSSW_10_6_20/src/run3_llp_analyzer/output_nTuples/MC/MC_Summer22/DYto2Mu_MLL-50to120_keepMDSHits_Merged/DYto2Mu_MLL-50to120_keepMDSHits_7311pb_weighted.root",
            "2022EE":"/storage/af/user/aalbert/CMSSW_10_6_20/src/run3_llp_analyzer/output_nTuples/MC/MC_Summer22EE/DYto2Mu_MLL-50to120_keepMDSHits_Merged/DYto2Mu_MLL-50to120_keepMDSHits_26642pb_weighted.root",
            "2023":"/storage/af/user/aalbert/CMSSW_10_6_20/src/run3_llp_analyzer/output_nTuples/MC/MC_Summer23/DYto2Mu_MLL-50to120_Merged/DYto2Mu_MLL-50to120_18411pb_weighted.root",
            "2023BPix":"/storage/af/user/aalbert/CMSSW_10_6_20/src/run3_llp_analyzer/output_nTuples/MC/MC_Summer23BPix/DYto2Mu_MLL-50to120_Merged/DYto2Mu_MLL-50to120_9451pb_weighted.root",
           "2024":"/storage/af/user/aalbert/CMSSW_10_6_20/src/run3_llp_analyzer/output_nTuples/MC/MC_Summer24/DYto2Mu_MLL-50to120_Merged/DYto2Mu_MLL-50to120_108753pb_weighted.root"}

data_path_lists = {"2022":["/storage/af/user/aalbert/CMSSW_10_6_20/src/run3_llp_analyzer/output_nTuples/Data/2022_Merged/Muon_Run2022C_PromptReco-v1_goodLumi.root",
                          "/storage/af/user/aalbert/CMSSW_10_6_20/src/run3_llp_analyzer/output_nTuples/Data/2022_Merged/Muon_Run2022D_PromptReco-v1_goodLumi.root"],
                 
                   "2022EE":["/storage/af/user/aalbert/CMSSW_10_6_20/src/run3_llp_analyzer/output_nTuples/Data/2022_Merged/Muon_Run2022E_PromptReco-v1_goodLumi.root",
                          "/storage/af/user/aalbert/CMSSW_10_6_20/src/run3_llp_analyzer/output_nTuples/Data/2022_Merged/Muon_Run2022F_PromptReco-v1_goodLumi.root",
                            "/storage/af/user/aalbert/CMSSW_10_6_20/src/run3_llp_analyzer/output_nTuples/Data/2022_Merged/Muon_Run2022G_PromptReco-v1_goodLumi.root"],
                   
                    "2023":["/storage/af/user/aalbert/CMSSW_10_6_20/src/run3_llp_analyzer/output_nTuples/Data/2023_Merged/Muon0_Run2023B_PromptReco-v1_goodLumi.root",
                     "/storage/af/user/aalbert/CMSSW_10_6_20/src/run3_llp_analyzer/output_nTuples/Data/2023_Merged/Muon1_Run2023B_PromptReco-v1_goodLumi.root",
                     "/storage/af/user/aalbert/CMSSW_10_6_20/src/run3_llp_analyzer/output_nTuples/Data/2023_Merged/Muon0_Run2023C_PromptReco-v1_goodLumi.root",
                     "/storage/af/user/aalbert/CMSSW_10_6_20/src/run3_llp_analyzer/output_nTuples/Data/2023_Merged/Muon1_Run2023C_PromptReco-v1_goodLumi.root",
                     "/storage/af/user/aalbert/CMSSW_10_6_20/src/run3_llp_analyzer/output_nTuples/Data/2023_Merged/Muon0_Run2023C_PromptReco-v2_goodLumi.root",
                     "/storage/af/user/aalbert/CMSSW_10_6_20/src/run3_llp_analyzer/output_nTuples/Data/2023_Merged/Muon1_Run2023C_PromptReco-v2_goodLumi.root",
                     "/storage/af/user/aalbert/CMSSW_10_6_20/src/run3_llp_analyzer/output_nTuples/Data/2023_Merged/Muon0_Run2023C_PromptReco-v3_goodLumi.root",
                     "/storage/af/user/aalbert/CMSSW_10_6_20/src/run3_llp_analyzer/output_nTuples/Data/2023_Merged/Muon1_Run2023C_PromptReco-v3_goodLumi.root",
                     "/storage/af/user/aalbert/CMSSW_10_6_20/src/run3_llp_analyzer/output_nTuples/Data/2023_Merged/Muon0_Run2023C_PromptReco-v4_goodLumi.root",
                     "/storage/af/user/aalbert/CMSSW_10_6_20/src/run3_llp_analyzer/output_nTuples/Data/2023_Merged/Muon1_Run2023C_PromptReco-v4_goodLumi.root"],
                  
                      "2023BPix":["/storage/af/user/aalbert/CMSSW_10_6_20/src/run3_llp_analyzer/output_nTuples/Data/2023_Merged/Muon0_Run2023D_PromptReco-v1_goodLumi.root",
                     "/storage/af/user/aalbert/CMSSW_10_6_20/src/run3_llp_analyzer/output_nTuples/Data/2023_Merged/Muon1_Run2023D_PromptReco-v1_goodLumi.root",
                     "/storage/af/user/aalbert/CMSSW_10_6_20/src/run3_llp_analyzer/output_nTuples/Data/2023_Merged/Muon0_Run2023D_PromptReco-v2_goodLumi.root",
                     "/storage/af/user/aalbert/CMSSW_10_6_20/src/run3_llp_analyzer/output_nTuples/Data/2023_Merged/Muon1_Run2023D_PromptReco-v2_goodLumi.root"],
                       
                    "2024":["/storage/af/user/aalbert/CMSSW_10_6_20/src/run3_llp_analyzer/output_nTuples/Data/2024_Merged/Muon0-Run2024B-PromptReco-v1-AOD_goodLumi.root",
                    "/storage/af/user/aalbert/CMSSW_10_6_20/src/run3_llp_analyzer/output_nTuples/Data/2024_Merged/Muon0-Run2024C-PromptReco-v1-AOD_goodLumi.root",
                    "/storage/af/user/aalbert/CMSSW_10_6_20/src/run3_llp_analyzer/output_nTuples/Data/2024_Merged/Muon0-Run2024D-PromptReco-v1-AOD_goodLumi.root",
                    "/storage/af/user/aalbert/CMSSW_10_6_20/src/run3_llp_analyzer/output_nTuples/Data/2024_Merged/Muon0-Run2024E-PromptReco-v1-AOD_goodLumi.root",
                    "/storage/af/user/aalbert/CMSSW_10_6_20/src/run3_llp_analyzer/output_nTuples/Data/2024_Merged/Muon0-Run2024E-PromptReco-v2-AOD_goodLumi.root",
                    "/storage/af/user/aalbert/CMSSW_10_6_20/src/run3_llp_analyzer/output_nTuples/Data/2024_Merged/Muon0-Run2024F-PromptReco-v1-AOD_goodLumi.root",
                    "/storage/af/user/aalbert/CMSSW_10_6_20/src/run3_llp_analyzer/output_nTuples/Data/2024_Merged/Muon0-Run2024G-PromptReco-v1-AOD_goodLumi.root",
                    "/storage/af/user/aalbert/CMSSW_10_6_20/src/run3_llp_analyzer/output_nTuples/Data/2024_Merged/Muon0-Run2024H-PromptReco-v1-AOD_goodLumi.root",
                    "/storage/af/user/aalbert/CMSSW_10_6_20/src/run3_llp_analyzer/output_nTuples/Data/2024_Merged/Muon0-Run2024I-PromptReco-v1-AOD_goodLumi.root",
                    "/storage/af/user/aalbert/CMSSW_10_6_20/src/run3_llp_analyzer/output_nTuples/Data/2024_Merged/Muon0-Run2024I-PromptReco-v2-AOD_goodLumi.root",
                    "/storage/af/user/aalbert/CMSSW_10_6_20/src/run3_llp_analyzer/output_nTuples/Data/2024_Merged/Muon1-Run2024B-PromptReco-v1-AOD_goodLumi.root",
                    "/storage/af/user/aalbert/CMSSW_10_6_20/src/run3_llp_analyzer/output_nTuples/Data/2024_Merged/Muon1-Run2024C-PromptReco-v1-AOD_goodLumi.root",
                    "/storage/af/user/aalbert/CMSSW_10_6_20/src/run3_llp_analyzer/output_nTuples/Data/2024_Merged/Muon1-Run2024D-PromptReco-v1-AOD_goodLumi.root",
                    "/storage/af/user/aalbert/CMSSW_10_6_20/src/run3_llp_analyzer/output_nTuples/Data/2024_Merged/Muon1-Run2024E-PromptReco-v1-AOD_goodLumi.root",
                    "/storage/af/user/aalbert/CMSSW_10_6_20/src/run3_llp_analyzer/output_nTuples/Data/2024_Merged/Muon1-Run2024E-PromptReco-v2-AOD_goodLumi.root",
                    "/storage/af/user/aalbert/CMSSW_10_6_20/src/run3_llp_analyzer/output_nTuples/Data/2024_Merged/Muon1-Run2024F-PromptReco-v1-AOD_goodLumi.root",
                    "/storage/af/user/aalbert/CMSSW_10_6_20/src/run3_llp_analyzer/output_nTuples/Data/2024_Merged/Muon1-Run2024G-PromptReco-v1-AOD_goodLumi.root",
                    "/storage/af/user/aalbert/CMSSW_10_6_20/src/run3_llp_analyzer/output_nTuples/Data/2024_Merged/Muon1-Run2024H-PromptReco-v1-AOD_goodLumi.root",
                    "/storage/af/user/aalbert/CMSSW_10_6_20/src/run3_llp_analyzer/output_nTuples/Data/2024_Merged/Muon1-Run2024I-PromptReco-v1-AOD_goodLumi.root",
                    "/storage/af/user/aalbert/CMSSW_10_6_20/src/run3_llp_analyzer/output_nTuples/Data/2024_Merged/Muon1-Run2024I-PromptReco-v2-AOD_goodLumi.root"]
                      
                             }

In [5]:
JSON_lists_base = "Data_MC_ClusterSize_JSONs_DTs"
JSON_lists_base_highMET = "Data_MC_ClusterSize_JSONs_highMET_DTs"

In [6]:
events_MC_full_dict= {}
for campaign, MC_path in MC_paths.items():
    print(campaign)
    #if "2022"!=campaign:continue
    events_MC_full = getLZDF(MC_path)
    events_MC_full = events_MC_full[events_MC_full.nDtRechitClusters>0]
    events_MC_full = events_MC_full[np.logical_and(events_MC_full.ZMass>50, events_MC_full.ZMass<120)]
    events_MC_full = events_MC_full[events_MC_full.Flag_all]
    events_MC_full = events_MC_full[events_MC_full.Flag_ecalBadCalibFilter]
    events_MC_full = events_MC_full[events_MC_full.jetVeto]
    events_MC_full_dict[campaign] = events_MC_full
    
events_data_full_dict= {}
for campaign, data_path_list in data_path_lists.items():
    print(campaign)
    #if "2022"!=campaign:continue
    data_events = [getLZDF(data_path) for data_path in data_path_list]
    events_data_full = ak.concatenate(data_events, axis=0)
    events_data_full = events_data_full[events_data_full.nDtRechitClusters>0]
    events_data_full = events_data_full[np.logical_and(events_data_full.ZMass>50, events_data_full.ZMass<120)]
    events_data_full = events_data_full[events_data_full.Flag_all]
    events_data_full = events_data_full[events_data_full.Flag_ecalBadCalibFilter]
    events_data_full = events_data_full[events_data_full.jetVeto]
    events_data_full_dict[campaign] = events_data_full

2022
2022EE
2023
2023BPix
2024
2022
2022EE
2023
2023BPix
2024


In [7]:
MC_kFactors = {"2022":0.9202108866645403,"2022EE":0.9205275233052539,"2023":0.9287192347533128,"2023BPix":0.938125553192627,"2024":0.9070469152942339}

### modify the input ntuples so that each entry corresponds with a cluster. As a result, some entries will be repeated twice (tne ones denoted "branch names") if there are two clusters in the event. At this step, all of the branches that we compute for the measurement should be included

In [8]:
# #define cluster level csc branches needed
# dt_branches = []
# for branch_name in events_MC_full.fields: 
#     if "dt" in branch_name and "csc" not in branch_name and "LLP" not in branch_name and "DNN" not in branch_name:
#         dt_branches.append(branch_name)


#event-level branches        
branch_names = ["weight", "pileupWeight", "ZMass"]


In [9]:
dt_branches = ["dtRechitClusterNHitStation1","dtRechitClusterSize", "dtRechitCluster_match_RPChits_dPhi0p5", "dtRechitCluster_match_RPCBx_dPhi0p5",
              "dtRechitCluster_matchToMuon1", "dtRechitCluster_matchToMuon2"]

In [10]:
def getSF(campaign, pt, eta):
    if campaign=="2022":
        MC_SF_LooseID = getMuonScaleFactor.getLooseIDEffArr_preEE(pt, eta)
        MC_SF_LooseISO = getMuonScaleFactor.getLooseISOEffArr_preEE(pt, eta)
        MC_SF_TightID = getMuonScaleFactor.getTightIDEffArr_preEE(pt, eta)
        MC_SF_TightISO = getMuonScaleFactor.getTightISOEffArr_preEE(pt, eta)
        MC_SF_HLT = getMuonScaleFactor.getHLTEffArr_preEE(pt, eta)
    elif campaign=="2022EE":
        MC_SF_LooseID = getMuonScaleFactor.getLooseIDEffArr_EE(pt, eta)
        MC_SF_LooseISO = getMuonScaleFactor.getLooseISOEffArr_EE(pt, eta)
        MC_SF_TightID = getMuonScaleFactor.getTightIDEffArr_EE(pt, eta)
        MC_SF_TightISO = getMuonScaleFactor.getTightISOEffArr_EE(pt, eta)
        MC_SF_HLT = getMuonScaleFactor.getHLTEffArr_EE(pt, eta)
    elif campaign=="2023":
        MC_SF_LooseID = getMuonScaleFactor.getLooseIDEffArr_preBPix(pt, eta)
        MC_SF_LooseISO = getMuonScaleFactor.getLooseISOEffArr_preBPix(pt, eta)
        MC_SF_TightID = getMuonScaleFactor.getTightIDEffArr_preBPix(pt, eta)
        MC_SF_TightISO = getMuonScaleFactor.getTightISOEffArr_preBPix(pt, eta)
        MC_SF_HLT = getMuonScaleFactor.getHLTEffArr_preBPix(pt, eta)
    elif campaign=="2023BPix" or campaign=="2024":
        MC_SF_LooseID = getMuonScaleFactor.getLooseIDEffArr_BPix(pt, eta)
        MC_SF_LooseISO = getMuonScaleFactor.getLooseISOEffArr_BPix(pt, eta)
        MC_SF_TightID = getMuonScaleFactor.getTightIDEffArr_BPix(pt, eta)
        MC_SF_TightISO = getMuonScaleFactor.getTightISOEffArr_BPix(pt, eta)
        MC_SF_HLT = getMuonScaleFactor.getHLTEffArr_BPix(pt, eta)
    else:
        print("invalid campaign. exiting...")
        exit()
    return MC_SF_LooseID, MC_SF_LooseISO, MC_SF_TightID, MC_SF_TightISO, MC_SF_HLT

In [11]:
#make more useful input awkward array, with all information in cluster-level format
def getClusterBranches(LZDF, campaign, isMC=False):
    new_df = ak.zip({field: ak.flatten(LZDF[field]) for field in dt_branches})
    if isMC:
        print("adding normalization branches for overall event counts and cluster size distribution")
        with open(JSON_lists_base+"/"+campaign+"/forwardVeto_eventCounts.json", 'r') as f:overallNormDict=json.load(f)
        #with open(JSON_lists_base+"/"+campaign+"/clusterSize_bins.json", 'r') as f:clusterSizeBinsDict=json.load(f)
        #with open(JSON_lists_base+"/"+campaign+"/clusterSize_shape_scaleFactors.json", 'r') as f:scaleFactorsDict=json.load(f)
        normValue = overallNormDict["data_counts"]/overallNormDict["MC_counts"]
        print(normValue)
        new_df = ak.with_field(new_df, ak.Array([normValue]*ak.count_nonzero(new_df.dtRechitClusterSize)), "overallNormalizationToData")
#         clusterSizeScaleFactor = ak.Array([1]*ak.count_nonzero(new_df.dtRechitClusterSize))
#         for binIndex, binsTuple in clusterSizeBinsDict.items():
#             clusterSizeScaleFactor = ak.where(np.logical_and(new_df.dtRechitClusterSize>=binsTuple[0],new_df.dtRechitClusterSize<binsTuple[1]), scaleFactorsDict[binIndex], clusterSizeScaleFactor)
#         new_df = ak.with_field(new_df, clusterSizeScaleFactor, "clusterSizeScaleFactor")
        with open(JSON_lists_base_highMET+"/"+campaign+"/forwardVeto_eventCounts.json", 'r') as f:overallNormDict=json.load(f)
        #with open(JSON_lists_base_highMET+"/"+campaign+"/clusterSize_bins.json", 'r') as f:clusterSizeBinsDict=json.load(f)
        #with open(JSON_lists_base_highMET+"/"+campaign+"/clusterSize_shape_scaleFactors.json", 'r') as f:scaleFactorsDict=json.load(f)
        normValue = overallNormDict["data_counts"]/overallNormDict["MC_counts"]
        print(normValue)
        new_df = ak.with_field(new_df, ak.Array([normValue]*ak.count_nonzero(new_df.dtRechitClusterSize)), "overallNormalizationToData_highMET")
#         clusterSizeScaleFactor = ak.Array([1]*ak.count_nonzero(new_df.dtRechitClusterSize))
#         for binIndex, binsTuple in clusterSizeBinsDict.items():
#             clusterSizeScaleFactor = ak.where(np.logical_and(new_df.dtRechitClusterSize>=binsTuple[0],new_df.dtRechitClusterSize<binsTuple[1]), scaleFactorsDict[binIndex], clusterSizeScaleFactor)
#         new_df = ak.with_field(new_df, clusterSizeScaleFactor, "clusterSizeScaleFactor_highMET")
#     compute cluster max chamber
#     hits_by_chamber = np.stack([ak.flatten(LZDF[branch]) for branch in csc_chamber_hit_branches], axis=1)
#     print(hits_by_chamber)
#     print(np.array(hits_by_chamber).shape)
#     maxBranchIndex = np.argmax(hits_by_chamber, axis=1)
#     #print(maxBranchIndex)
#     chamber_masks_lists = []
#     for chamber_index in forward_chamber_field_indices:
#         chamber_masks_lists.append((maxBranchIndex==chamber_index))
        #print(np.where(maxBranchIndex==chamber_index))
    #print(np.stack(chamber_masks_lists, axis=1))
    #new_df = ak.with_field(new_df, np.any(np.stack(chamber_masks_lists, axis=1), axis=1), "forward_max_chamber")
    
    print("finished dt branches")
#     newDNN = ak.flatten(ak.mask(LZDF["cscRechitClusterDNN_bkgMC_plusBeamHalo"], LZDF["cscRechitClusterDNN_bkgMC_plusBeamHalo"]>0))
#     newDNN = newDNN[~ak.is_none(newDNN)]
#     new_df = ak.with_field(new_df, newDNN, "cscRechitClusterDNN_bkgMC_plusBeamHalo")
    
    for branch in branch_names:
        if (not isMC) and (branch in ["weight", "pileupWeight"]):
            continue
        new_df = ak.with_field(new_df, np.repeat(LZDF[branch],LZDF["nDtRechitClusters"]), branch)

    
    column_indices_probe = np.array(ak.flatten(ak.values_astype(LZDF["dtRechitCluster_matchToMuon2"], int)))
    column_indices_tag = np.array(ak.flatten(ak.values_astype(LZDF["dtRechitCluster_matchToMuon1"], int)))
    row_indices = np.arange(np.size(column_indices_probe), dtype=int)
    
    #compute 
    if isMC:
        MC_SF_LooseID, MC_SF_LooseISO, MC_SF_TightID, MC_SF_TightISO, MC_SF_HLT = getSF(campaign, np.array(LZDF.lepPt), np.array(LZDF.lepEta))
        
        
        MC_SF_LooseID = np.repeat(MC_SF_LooseID,np.array(LZDF["nDtRechitClusters"]), axis=0)[row_indices,column_indices_probe]
        MC_SF_LooseISO = np.repeat(np.array(MC_SF_LooseISO),np.array(LZDF["nDtRechitClusters"]), axis=0)[row_indices,column_indices_probe]
        MC_SF_TightID = np.repeat(MC_SF_TightID,np.array(LZDF["nDtRechitClusters"]), axis=0)[row_indices,column_indices_tag]
        MC_SF_TightISO = np.repeat(MC_SF_TightISO,np.array(LZDF["nDtRechitClusters"]), axis=0)[row_indices,column_indices_tag]
        MC_SF_HLT = np.repeat(MC_SF_HLT,np.array(LZDF["nDtRechitClusters"]), axis=0)[row_indices,column_indices_tag]
        
        MC_Weight_Total = new_df["weight"]*new_df["pileupWeight"]*MC_SF_LooseID*MC_SF_LooseISO*MC_SF_TightID*MC_SF_TightISO*MC_SF_HLT*MC_kFactors[campaign]
        new_df = ak.with_field(new_df, MC_Weight_Total, "weight_total")
    
    print("at muon variables")
    
#     #load pT, eta, and phi for tag and probe muons
    probe_pT = np.repeat(np.array(LZDF["lepPt"]),np.array(LZDF["nDtRechitClusters"]), axis=0)[row_indices,column_indices_probe]
#     probe_eta = np.repeat(np.array(LZDF["lepEta"]),np.array(LZDF["nDtRechitClusters"]), axis=0)[row_indices,column_indices_probe]
#     probe_phi = np.repeat(np.array(LZDF["lepPhi"]),np.array(LZDF["nDtRechitClusters"]), axis=0)[row_indices,column_indices_probe]
    
# #     tag_pT = np.repeat(np.array(LZDF["lepPt"]),np.array(LZDF["nCscRechitClusters"]), axis=0)[row_indices,column_indices_tag]
# #     tag_eta = np.repeat(np.array(LZDF["lepEta"]),np.array(LZDF["nCscRechitClusters"]), axis=0)[row_indices,column_indices_tag]
# #     tag_phi = np.repeat(np.array(LZDF["lepPhi"]),np.array(LZDF["nCscRechitClusters"]), axis=0)[row_indices,column_indices_tag]
    
    new_df = ak.with_field(new_df, probe_pT, "probe_pT")
#     new_df = ak.with_field(new_df, probe_eta, "probe_eta")
#     new_df = ak.with_field(new_df, probe_phi, "probe_phi")
    
# #     new_df = ak.with_field(new_df, tag_pT, "tag_pT")
# #     new_df = ak.with_field(new_df, tag_eta, "tag_eta")
# #     new_df = ak.with_field(new_df, tag_phi, "tag_phi")
    
#      #deltaR(cluster, muon)
#     new_df = ak.with_field(new_df, np.sqrt((new_df["dtRechitClusterEta"]-new_df["probe_eta"])**2+(new_df["dtRechitClusterPhi"]-new_df["probe_phi"])**2), "dtRechitClusterMuonDeltaR")
    
#     #DNN inputs - hit fractions in stations/rings
#     new_df = ak.with_field(new_df, (new_df.cscRechitClusterNRechitChamberPlus11+new_df.cscRechitClusterNRechitChamberMinus11+new_df.cscRechitClusterNRechitChamberPlus12+new_df.cscRechitClusterNRechitChamberMinus12+new_df.cscRechitClusterNRechitChamberPlus13+new_df.cscRechitClusterNRechitChamberMinus13)/new_df.cscRechitClusterSize, "cscRechitClusterFracS1")
#     new_df = ak.with_field(new_df, (new_df.cscRechitClusterNRechitChamberPlus21+new_df.cscRechitClusterNRechitChamberMinus21+new_df.cscRechitClusterNRechitChamberPlus22+new_df.cscRechitClusterNRechitChamberMinus22)/new_df.cscRechitClusterSize, "cscRechitClusterFracS2")
#     new_df = ak.with_field(new_df, (new_df.cscRechitClusterNRechitChamberPlus31+new_df.cscRechitClusterNRechitChamberMinus31+new_df.cscRechitClusterNRechitChamberPlus32+new_df.cscRechitClusterNRechitChamberMinus32)/new_df.cscRechitClusterSize, "cscRechitClusterFracS3")
#     new_df = ak.with_field(new_df, (new_df.cscRechitClusterNRechitChamberPlus41+new_df.cscRechitClusterNRechitChamberMinus41+new_df.cscRechitClusterNRechitChamberPlus42+new_df.cscRechitClusterNRechitChamberMinus42)/new_df.cscRechitClusterSize, "cscRechitClusterFracS4")

#     new_df = ak.with_field(new_df,(new_df.cscRechitClusterNRechitChamberPlus11+new_df.cscRechitClusterNRechitChamberMinus11+new_df.cscRechitClusterNRechitChamberPlus21+new_df.cscRechitClusterNRechitChamberMinus21+new_df.cscRechitClusterNRechitChamberPlus31+new_df.cscRechitClusterNRechitChamberMinus31+new_df.cscRechitClusterNRechitChamberPlus41+new_df.cscRechitClusterNRechitChamberMinus41)/new_df.cscRechitClusterSize, "cscRechitClusterFracR1")
#     new_df = ak.with_field(new_df, (new_df.cscRechitClusterNRechitChamberPlus12+new_df.cscRechitClusterNRechitChamberMinus12+new_df.cscRechitClusterNRechitChamberPlus22+new_df.cscRechitClusterNRechitChamberMinus22+new_df.cscRechitClusterNRechitChamberPlus32+new_df.cscRechitClusterNRechitChamberMinus32+new_df.cscRechitClusterNRechitChamberPlus42+new_df.cscRechitClusterNRechitChamberMinus42)/new_df.cscRechitClusterSize, "cscRechitClusterFracR2")
#     new_df = ak.with_field(new_df, (new_df.cscRechitClusterNRechitChamberPlus13+new_df.cscRechitClusterNRechitChamberMinus13)/new_df.cscRechitClusterSize, "cscRechitClusterFracR3")
    
    #forward hits branch
    #new_df = ak.with_field(new_df, new_df.cscRechitClusterNRechitChamberPlus11+new_df.cscRechitClusterNRechitChamberMinus11+new_df.cscRechitClusterNRechitChamberPlus12 + new_df.cscRechitClusterNRechitChamberMinus12, "forward_hits")
    return new_df

In [12]:
events_MC_dict = {}; events_data_dict = {}
for campaign in list(events_MC_full_dict.keys()):
    #if campaign!="2023":continue
    print(campaign)
    print("MC")
    events_MC = getClusterBranches(events_MC_full_dict[campaign], campaign, True)
    print("now data")
    events_data = getClusterBranches(events_data_full_dict[campaign], campaign, False)
    events_MC_dict[campaign] = events_MC
    events_data_dict[campaign] = events_data

2022
MC
adding normalization branches for overall event counts and cluster size distribution
0.5641598090937581
2.2088988191001695
finished dt branches
at muon variables
now data
finished dt branches
at muon variables
2022EE
MC
adding normalization branches for overall event counts and cluster size distribution
0.5128216095105117
2.4249322740397456
finished dt branches
at muon variables
now data
finished dt branches
at muon variables
2023
MC
adding normalization branches for overall event counts and cluster size distribution
0.5289330944956476
2.3431722643930697
finished dt branches
at muon variables
now data
finished dt branches
at muon variables
2023BPix
MC
adding normalization branches for overall event counts and cluster size distribution
0.5156576020489922
2.431960881218399
finished dt branches
at muon variables
now data
finished dt branches
at muon variables
2024
MC
adding normalization branches for overall event counts and cluster size distribution
0.2719324052491383
1.463605006

### Code to Mask Data According to Specific Cuts - Low MET and High MET, along with cutflows

In [13]:
def makeForwardVetoMask(events, mask, forwardVetoList: list=[]):
    forwardMask = mask
    if "forward_veto" in forwardVetoList:
        forwardMask = ak.mask(forwardMask, events.dtRechitClusterNHitStation1==0)
    if "forward_veto_highMET" in forwardVetoList:
        forwardMask = ak.mask(forwardMask, events.dtRechitClusterNHitStation1/events.dtRechitClusterSize<0.9)
    return forwardMask

In [14]:
def makeEventMask(events, noMaskList: list=[], forwardVetoMaskList: list=[], noCuts=False, invertHotspot=False, clusterSize=100):
    mask = events.dtRechitCluster_matchToMuon1|events.dtRechitCluster_matchToMuon2
    mask = makeForwardVetoMask(events, mask, forwardVetoMaskList)
    #hotspotMask = np.all([events.dtRechitClusterPhi>0.8,events.dtRechitClusterPhi<2.1, np.abs(events.dtRechitClusterEta)>0.2, np.abs(events.dtRechitClusterEta)<0.4], axis=0)
    #mask out hotspot automatically
    #mask=ak.mask(mask, np.logical_or(np.logical_and(np.logical_or(events.cscRechitClusterPhi<-0.3,events.cscRechitClusterPhi>0.6),abs(events.cscRechitClusterPhi)<2.8), events.cscRechitClusterEta>-1.9))
    if invertHotspot:hotspotMask=np.logical_not(hotspotMask)
    #mask=ak.mask(mask, np.logical_not(hotspotMask))
    if noCuts:
        return mask
    if "RPC_veto" not in noMaskList:
        mask = ak.mask(mask, events.dtRechitCluster_match_RPChits_dPhi0p5>0)
    if "RPCBX_veto" not in noMaskList:
        mask = ak.mask(mask, events.dtRechitCluster_match_RPCBx_dPhi0p5==0)
    if "clusterSize_veto" not in noMaskList:
        mask = ak.mask(mask, events.dtRechitClusterSize>clusterSize)
    return mask

In [15]:
def makeEventMaskHighMET(events, noMaskList: list=[], forwardVetoMaskList: list=[], noCuts=False, invertHotspot=False, clusterSize=130):
    mask = events.dtRechitCluster_matchToMuon1|events.dtRechitCluster_matchToMuon2
    mask = makeForwardVetoMask(events, mask, forwardVetoMaskList)
    #hotspotMask = np.all([events.dtRechitClusterPhi>0.8,events.dtRechitClusterPhi<2.1, np.abs(events.dtRechitClusterEta)>0.2, np.abs(events.dtRechitClusterEta)<0.4], axis=0)
    #mask out hotspot automatically
    #mask=ak.mask(mask, np.logical_or(np.logical_and(np.logical_or(events.cscRechitClusterPhi<-0.3,events.cscRechitClusterPhi>0.6),abs(events.cscRechitClusterPhi)<2.8), events.cscRechitClusterEta>-1.9))
    if invertHotspot:hotspotMask=np.logical_not(hotspotMask)
    #mask=ak.mask(mask, np.logical_not(hotspotMask))
    if noCuts:
        return mask
    if "RPC_veto" not in noMaskList:
        mask = ak.mask(mask, events.dtRechitCluster_match_RPChits_dPhi0p5>0)
    if "RPCBX_veto" not in noMaskList:
        mask = ak.mask(mask, events.dtRechitCluster_match_RPCBx_dPhi0p5==0)
    if "clusterSize_veto" not in noMaskList:
        mask = ak.mask(mask, events.dtRechitClusterSize>clusterSize)
    return mask

#### compute efficiencies (no cuts applied other than forward veto or high MET equivalent, except for measurement of forward veto efficiency itself)


In [16]:
def getEff(numerator, denominator):
    num = rt.TH1D("num","num",1,0,1)
    denom = rt.TH1D("denom","denom",1,0,1)
    num.SetBinContent(1,numerator)
    denom.SetBinContent(1,denominator)
    teff = rt.TEfficiency(num, denom)
    return (teff.GetEfficiency(1),teff.GetEfficiencyErrorLow(1),teff.GetEfficiencyErrorUp(1),teff)

In [17]:
# data efficiencies
data_lowMET_counts_efficiencies = {}
for campaign in list(events_data_dict.keys()):
    counts_efficiencies_dict={}
    print("###################################")
    print("###################################")
    print(f"Computing Efficiencies for {campaign}")
    events_data = events_data_dict[campaign]
    print("computing low MET efficiencies in Data")

    denom = ak.count_nonzero(makeEventMask(events_data, [], [], True))
    print("Data Denominator: ", denom)
    counts_efficiencies_dict["Events_Total"] = denom

    num_forward = ak.count_nonzero(makeEventMask(events_data, [], ["forward_veto"], True))
    print("Forward Veto Efficiency: ", num_forward/denom*100)
    counts_efficiencies_dict["Pass_Forward"] = num_forward
    counts_efficiencies_dict["Forward_Eff"] = getEff(num_forward, denom)
    
    print("events passing forward veto: ", num_forward)

    num_RPC = ak.count_nonzero(makeEventMask(events_data, ['clusterSize_veto','RPCBX_veto'],["forward_veto"]))
    print("RPC Veto Efficiency: ", num_RPC/num_forward*100)
    counts_efficiencies_dict["RPC_Eff"] = getEff(num_RPC, num_forward)
    
    num_RPXBX = ak.count_nonzero(makeEventMask(events_data, ['RPC_veto','clusterSize_veto'],["forward_veto"]))
    print("RPCBX Veto Efficiency: ", num_RPXBX/num_forward*100)
    counts_efficiencies_dict["RPCBX_Eff"] = getEff(num_RPXBX, num_forward)

    num_clusterSize = ak.count_nonzero(makeEventMask(events_data, ['RPC_veto','RPCBX_veto'],["forward_veto"],clusterSize=120))
    print("ClusterSize 120 Veto Efficiency: ", num_clusterSize/num_forward*100)
    counts_efficiencies_dict["clusterSize120_Eff"] = getEff(num_clusterSize, num_forward)
    
    num_clusterSize = ak.count_nonzero(makeEventMask(events_data, ['RPC_veto','RPCBX_veto'],["forward_veto"],clusterSize=160))
    print("ClusterSize 160 Veto Efficiency: ", num_clusterSize/num_forward*100)
    counts_efficiencies_dict["clusterSize160_Eff"] = getEff(num_clusterSize, num_forward)
    
    data_lowMET_counts_efficiencies[campaign] = counts_efficiencies_dict
    

###################################
###################################
Computing Efficiencies for 2022
computing low MET efficiencies in Data
Data Denominator:  1369839
Forward Veto Efficiency:  0.6953371892609277
events passing forward veto:  9525
RPC Veto Efficiency:  97.42782152230971
RPCBX Veto Efficiency:  88.20997375328083
ClusterSize 120 Veto Efficiency:  1.2703412073490814
ClusterSize 160 Veto Efficiency:  0.12598425196850394
###################################
###################################
Computing Efficiencies for 2022EE
computing low MET efficiencies in Data
Data Denominator:  5131480
Forward Veto Efficiency:  0.5611051782331803
events passing forward veto:  28793
RPC Veto Efficiency:  97.5237036779773
RPCBX Veto Efficiency:  88.25756260202132
ClusterSize 120 Veto Efficiency:  1.3128190879727712
ClusterSize 160 Veto Efficiency:  0.10766505747924841
###################################
###################################
Computing Efficiencies for 2023
computing low ME

In [18]:
# MC efficiencies
MC_lowMET_counts_efficiencies = {}
for campaign in list(events_MC_dict.keys()):
    counts_efficiencies_dict={}
    print("###################################")
    print("###################################")
    print(f"Computing Efficiencies for {campaign}")
    events_MC = events_MC_dict[campaign]
    print("computing low MET efficiencies in MC")

    denom = ak.sum(ak.mask(events_MC.weight_total, makeEventMask(events_MC, [], [], True)))
    print("MC Denominator: ", denom)
    counts_efficiencies_dict["Events_Total"] = denom

    num_forward = ak.sum(ak.mask(events_MC.weight_total, makeEventMask(events_MC, [], ["forward_veto"], True)))
    print("Forward Veto Efficiency: ", num_forward/denom*100)
    counts_efficiencies_dict["Pass_Forward"] = num_forward
    counts_efficiencies_dict["Forward_Eff"] = getEff(num_forward, denom)
    
    print("events passing forward veto: ", num_forward)

    num_RPC = ak.sum(ak.mask(events_MC.weight_total, makeEventMask(events_MC, ['clusterSize_veto','RPCBX_veto'],["forward_veto"])))
    print("RPC Veto Efficiency: ", num_RPC/num_forward*100)
    counts_efficiencies_dict["RPC_Eff"] = getEff(num_RPC, num_forward)

    num_RPXBX = ak.sum(ak.mask(events_MC.weight_total, makeEventMask(events_MC, ['RPC_veto','clusterSize_veto'],["forward_veto"])))
    print("RPCBX Veto Efficiency: ", num_RPXBX/num_forward*100)
    counts_efficiencies_dict["RPCBX_Eff"] = getEff(num_RPXBX, num_forward)

    num_clusterSize = ak.sum(ak.mask(events_MC.weight_total, makeEventMask(events_MC, ['RPC_veto','RPCBX_veto'],["forward_veto"],clusterSize=120)))
    print("ClusterSize 120 Veto Efficiency: ", num_clusterSize/num_forward*100)
    counts_efficiencies_dict["clusterSize120_Eff"] = getEff(num_clusterSize, num_forward)
    
    num_clusterSize = ak.sum(ak.mask(events_MC.weight_total, makeEventMask(events_MC, ['RPC_veto','RPCBX_veto'],["forward_veto"],clusterSize=160)))
    print("ClusterSize 160 Veto Efficiency: ", num_clusterSize/num_forward*100)
    counts_efficiencies_dict["clusterSize160_Eff"] = getEff(num_clusterSize, num_forward)
    
    MC_lowMET_counts_efficiencies[campaign] = counts_efficiencies_dict

###################################
###################################
Computing Efficiencies for 2022
computing low MET efficiencies in MC
MC Denominator:  620145.6527366092
Forward Veto Efficiency:  2.722507827528773
events passing forward veto:  16883.513937833588
RPC Veto Efficiency:  99.77698108241847
RPCBX Veto Efficiency:  99.43783537910204
ClusterSize 120 Veto Efficiency:  5.208127120950558
ClusterSize 160 Veto Efficiency:  1.4884358427191486
###################################
###################################
Computing Efficiencies for 2022EE
computing low MET efficiencies in MC
MC Denominator:  2116133.326664567
Forward Veto Efficiency:  2.6532463347774984
events passing forward veto:  56146.22992873277
RPC Veto Efficiency:  99.8479680895342
RPCBX Veto Efficiency:  99.5087931821539
ClusterSize 120 Veto Efficiency:  4.686876982548749
ClusterSize 160 Veto Efficiency:  1.4016728447557265
###################################
###################################
Computing Effici

In [19]:
# data efficiencies
data_highMET_counts_efficiencies = {}
for campaign in list(events_data_dict.keys()):
    counts_efficiencies_dict={}
    print("###################################")
    print("###################################")
    print(f"Computing Efficiencies for {campaign}")
    events_data = events_data_dict[campaign]
    print("computing high MET efficiencies in Data")

    denom = ak.count_nonzero(makeEventMaskHighMET(events_data, [], [], True))
    print("Data Denominator: ", denom)
    counts_efficiencies_dict["Events_Total"] = denom

    num_forward = ak.count_nonzero(makeEventMaskHighMET(events_data, [], ["forward_veto_highMET"], True))
    print("Forward Veto Efficiency: ", num_forward/denom*100)
    counts_efficiencies_dict["Pass_Forward"] = num_forward
    counts_efficiencies_dict["Forward_Eff"] = getEff(num_forward, denom)

    num_RPC = ak.count_nonzero(makeEventMaskHighMET(events_data, ['clusterSize_veto','RPCBX_veto']))
    print("RPC Veto Efficiency: ", num_RPC/denom*100)
    counts_efficiencies_dict["RPC_Eff"] = getEff(num_RPC, denom)

    num_RPXBX = ak.count_nonzero(makeEventMaskHighMET(events_data, ['RPC_veto','clusterSize_veto']))
    print("RPCBX Veto Efficiency: ", num_RPXBX/denom*100)
    counts_efficiencies_dict["RPCBX_Eff"] = getEff(num_RPXBX, denom)

    num_clusterSize = ak.count_nonzero(makeEventMaskHighMET(events_data, ['RPC_veto','RPCBX_veto']))
    print("ClusterSize Veto Efficiency: ", num_clusterSize/denom*100)
    counts_efficiencies_dict["clusterSize_Eff"] = getEff(num_clusterSize, denom)
    
    data_highMET_counts_efficiencies[campaign] = counts_efficiencies_dict

###################################
###################################
Computing Efficiencies for 2022
computing high MET efficiencies in Data
Data Denominator:  1369839
Forward Veto Efficiency:  99.38153315827627
RPC Veto Efficiency:  98.2819878832476
RPCBX Veto Efficiency:  93.306001654209
ClusterSize Veto Efficiency:  0.8898855996945626
###################################
###################################
Computing Efficiencies for 2022EE
computing high MET efficiencies in Data
Data Denominator:  5131480
Forward Veto Efficiency:  99.49844879060232
RPC Veto Efficiency:  98.52282382470554
RPCBX Veto Efficiency:  93.1853383429342
ClusterSize Veto Efficiency:  0.7694076562707055
###################################
###################################
Computing Efficiencies for 2023
computing high MET efficiencies in Data
Data Denominator:  3858793
Forward Veto Efficiency:  99.41958534702432
RPC Veto Efficiency:  97.22099630635797
RPCBX Veto Efficiency:  92.26592356729164
ClusterSize V

In [20]:
# MC efficiencies
MC_highMET_counts_efficiencies = {}
for campaign in list(events_MC_dict.keys()):
    print("###################################")
    print("###################################")
    counts_efficiencies_dict={}
    print(f"Computing Efficiencies for {campaign}")
    events_MC = events_MC_dict[campaign]
    print("computing high MET efficiencies in MC")

    denom = ak.sum(ak.mask(events_MC.weight_total, makeEventMaskHighMET(events_MC, [], [], True)))
    print("MC Denominator: ", denom)
    counts_efficiencies_dict["Events_Total"] = denom

    num_forward = ak.sum(ak.mask(events_MC.weight_total, makeEventMaskHighMET(events_MC, [], ["forward_veto_highMET"], True)))
    print("Forward Veto Efficiency: ", num_forward/denom*100)
    counts_efficiencies_dict["Pass_Forward"] = num_forward
    counts_efficiencies_dict["Forward_Eff"] = getEff(num_forward, denom)

    num_RPC = ak.sum(ak.mask(events_MC.weight_total, makeEventMaskHighMET(events_MC, ['clusterSize_veto','RPCBX_veto'])))
    print("RPC Veto Efficiency: ", num_RPC/denom*100)
    counts_efficiencies_dict["RPC_Eff"] = getEff(num_RPC, denom)

    num_RPXBX = ak.sum(ak.mask(events_MC.weight_total, makeEventMaskHighMET(events_MC, ['RPC_veto','clusterSize_veto'])))
    print("RPCBX Veto Efficiency: ", num_RPXBX/denom*100)
    counts_efficiencies_dict["RPCBX_Eff"] = getEff(num_RPXBX, denom)

    num_clusterSize = ak.sum(ak.mask(events_MC.weight_total, makeEventMask(events_MC, ['RPC_veto','RPCBX_veto'])))
    print("ClusterSize Veto Efficiency: ", num_clusterSize/num_forward*100)
    counts_efficiencies_dict["clusterSize_Eff"] = getEff(num_clusterSize, denom)
    
    MC_highMET_counts_efficiencies[campaign] = counts_efficiencies_dict

###################################
###################################
Computing Efficiencies for 2022
computing high MET efficiencies in MC
MC Denominator:  620145.6527366092
Forward Veto Efficiency:  99.14836743970359
RPC Veto Efficiency:  99.71330968891967
RPCBX Veto Efficiency:  98.64473402983886
ClusterSize Veto Efficiency:  4.467136761928306
###################################
###################################
Computing Efficiencies for 2022EE
computing high MET efficiencies in MC
MC Denominator:  2116133.326664567
Forward Veto Efficiency:  99.24803496951164
RPC Veto Efficiency:  99.69646864184622
RPCBX Veto Efficiency:  98.44540014547117
ClusterSize Veto Efficiency:  4.310228302248629
###################################
###################################
Computing Efficiencies for 2023
computing high MET efficiencies in MC
MC Denominator:  1646824.2897196922
Forward Veto Efficiency:  99.17408970934272
RPC Veto Efficiency:  99.68041542757867
RPCBX Veto Efficiency:  98.3594921

In [21]:
def findEffErr(dataTuple, MCTuple):
    dataEff = dataTuple[0]; MCEff = MCTuple[0]
    dataAvgErr = (dataTuple[1]+dataTuple[2])/2; MCAvgErr = (MCTuple[1]+MCTuple[2])/2
    return (dataEff/MCEff)*((dataAvgErr/dataEff)**2+(MCAvgErr/MCEff)**2)**0.5

In [22]:
#compute uncertainties in Data/MC discrepancy
print("EFFICIENCY DISCREPANCIES - LOW MET DTs")
for campaign in list(data_lowMET_counts_efficiencies.keys()):
    print("##################################")
    print(campaign)
    print("# clusters pass forward veto (data/MC)", data_lowMET_counts_efficiencies[campaign]["Pass_Forward"], "/",MC_lowMET_counts_efficiencies[campaign]["Pass_Forward"])
    print("Data/MC ratio clusters that pass forward veto", data_lowMET_counts_efficiencies[campaign]["Pass_Forward"]/MC_lowMET_counts_efficiencies[campaign]["Pass_Forward"])
    print("Forward Veto Efficiency Discrepancy: ", data_lowMET_counts_efficiencies[campaign]["Forward_Eff"][0]/MC_lowMET_counts_efficiencies[campaign]["Forward_Eff"][0],"+-",findEffErr(data_lowMET_counts_efficiencies[campaign]["Forward_Eff"],MC_lowMET_counts_efficiencies[campaign]["Forward_Eff"]))
    print("Cluster Size 120 Efficiency Discrepancy: ", data_lowMET_counts_efficiencies[campaign]["clusterSize120_Eff"][0]/MC_lowMET_counts_efficiencies[campaign]["clusterSize120_Eff"][0],"+-",findEffErr(data_lowMET_counts_efficiencies[campaign]["clusterSize120_Eff"],MC_lowMET_counts_efficiencies[campaign]["clusterSize120_Eff"]))
    print("Cluster Size 160 Efficiency Discrepancy: ", data_lowMET_counts_efficiencies[campaign]["clusterSize160_Eff"][0]/MC_lowMET_counts_efficiencies[campaign]["clusterSize160_Eff"][0],"+-",findEffErr(data_lowMET_counts_efficiencies[campaign]["clusterSize160_Eff"],MC_lowMET_counts_efficiencies[campaign]["clusterSize160_Eff"]))
    print("RPC Hits Efficiency Discrepancy: ", data_lowMET_counts_efficiencies[campaign]["RPC_Eff"][0]/MC_lowMET_counts_efficiencies[campaign]["RPC_Eff"][0],"+-",findEffErr(data_lowMET_counts_efficiencies[campaign]["RPC_Eff"],MC_lowMET_counts_efficiencies[campaign]["RPC_Eff"]))
    print("RPCBX Efficiency Discrepancy: ", data_lowMET_counts_efficiencies[campaign]["RPCBX_Eff"][0]/MC_lowMET_counts_efficiencies[campaign]["RPCBX_Eff"][0],"+-",findEffErr(data_lowMET_counts_efficiencies[campaign]["RPCBX_Eff"],MC_lowMET_counts_efficiencies[campaign]["RPCBX_Eff"]))
    #print("NStation10 Efficiency Discrepancy: ", data_lowMET_counts_efficiencies[campaign]["NStation10_Eff"][0]/MC_lowMET_counts_efficiencies[campaign]["NStation10_Eff"][0],"+-",findEffErr(data_lowMET_counts_efficiencies[campaign]["NStation10_Eff"],MC_lowMET_counts_efficiencies[campaign]["NStation10_Eff"]))

EFFICIENCY DISCREPANCIES - LOW MET DTs
##################################
2022
# clusters pass forward veto (data/MC) 9525 / 16883.513937833588
Data/MC ratio clusters that pass forward veto 0.5641598090937581
Forward Veto Efficiency Discrepancy:  0.25540319195044775 +- 0.003264778791312209
Cluster Size 120 Efficiency Discrepancy:  0.24391516909004052 +- 0.02445141073263199
Cluster Size 160 Efficiency Discrepancy:  0.08464204391796265 +- 0.028635506580045704
RPC Hits Efficiency Discrepancy:  0.9764558965943428 +- 0.0017222776935768881
RPCBX Efficiency Discrepancy:  0.8870866246935534 +- 0.0034187301765915535
##################################
2022EE
# clusters pass forward veto (data/MC) 28793 / 56146.22992873277
Data/MC ratio clusters that pass forward veto 0.5128216095105117
Forward Veto Efficiency Discrepancy:  0.2114787348910951 +- 0.0015272328606393904
Cluster Size 120 Efficiency Discrepancy:  0.2801053010055436 +- 0.01564134955915004
Cluster Size 160 Efficiency Discrepancy:  0.076

In [23]:
#compute uncertainties in Data/MC discrepancy
print("EFFICIENCY DISCREPANCIES - HIGH MET DTs")
for campaign in list(data_highMET_counts_efficiencies.keys()):
    print("##################################")
    print(campaign)
    print("# clusters (data/MC)", data_highMET_counts_efficiencies[campaign]["Events_Total"], "/",MC_highMET_counts_efficiencies[campaign]["Events_Total"])
    print("Data/MC ratio clusters", data_highMET_counts_efficiencies[campaign]["Events_Total"]/MC_highMET_counts_efficiencies[campaign]["Events_Total"])
    print("# clusters pass forward veto (data/MC)", data_highMET_counts_efficiencies[campaign]["Pass_Forward"], "/",MC_highMET_counts_efficiencies[campaign]["Pass_Forward"])
    print("Forward Veto Efficiency Discrepancy: ", data_highMET_counts_efficiencies[campaign]["Forward_Eff"][0]/MC_highMET_counts_efficiencies[campaign]["Forward_Eff"][0],"+-",findEffErr(data_highMET_counts_efficiencies[campaign]["Forward_Eff"],MC_highMET_counts_efficiencies[campaign]["Forward_Eff"]))
    print("Cluster Size 110 Efficiency Discrepancy: ", data_highMET_counts_efficiencies[campaign]["clusterSize_Eff"][0]/MC_highMET_counts_efficiencies[campaign]["clusterSize_Eff"][0],"+-",findEffErr(data_highMET_counts_efficiencies[campaign]["clusterSize_Eff"],MC_highMET_counts_efficiencies[campaign]["clusterSize_Eff"]))
    #print("Cluster Size 130 Efficiency Discrepancy: ", data_highMET_counts_efficiencies[campaign]["clusterSize130_Eff"][0]/MC_highMET_counts_efficiencies[campaign]["clusterSize130_Eff"][0],"+-",findEffErr(data_highMET_counts_efficiencies[campaign]["clusterSize130_Eff"],MC_highMET_counts_efficiencies[campaign]["clusterSize130_Eff"]))
    print("RPC Hits Efficiency Discrepancy: ", data_highMET_counts_efficiencies[campaign]["RPC_Eff"][0]/MC_highMET_counts_efficiencies[campaign]["RPC_Eff"][0],"+-",findEffErr(data_highMET_counts_efficiencies[campaign]["RPC_Eff"],MC_highMET_counts_efficiencies[campaign]["RPC_Eff"]))
    print("RPCBX Efficiency Discrepancy: ", data_highMET_counts_efficiencies[campaign]["RPCBX_Eff"][0]/MC_highMET_counts_efficiencies[campaign]["RPCBX_Eff"][0],"+-",findEffErr(data_highMET_counts_efficiencies[campaign]["RPCBX_Eff"],MC_highMET_counts_efficiencies[campaign]["RPCBX_Eff"]))
    #print("NStation10 Efficiency Discrepancy: ", data_highMET_counts_efficiencies[campaign]["NStation10_Eff"][0]/MC_highMET_counts_efficiencies[campaign]["NStation10_Eff"][0],"+-",findEffErr(data_highMET_counts_efficiencies[campaign]["NStation10_Eff"],MC_highMET_counts_efficiencies[campaign]["NStation10_Eff"]))

EFFICIENCY DISCREPANCIES - HIGH MET DTs
##################################
2022
# clusters (data/MC) 1369839 / 620145.6527366092
Data/MC ratio clusters 2.2088988191001695
# clusters pass forward veto (data/MC) 1361367 / 614864.2904366415
Forward Veto Efficiency Discrepancy:  1.002351684900051 +- 0.00013683450967736982
Cluster Size 110 Efficiency Discrepancy:  0.200918238878168 +- 0.0021737932125344434
RPC Hits Efficiency Discrepancy:  0.9856456293534189 +- 0.00013073281115353545
RPCBX Efficiency Discrepancy:  0.9458791953961277 +- 0.00025895459517281315
##################################
2022EE
# clusters (data/MC) 5131480 / 2116133.326664567
Data/MC ratio clusters 2.4249322740397456
# clusters pass forward veto (data/MC) 5105743 / 2100220.7440495393
Forward Veto Efficiency Discrepancy:  1.0025231111242416 +- 6.797575189984509e-05
Cluster Size 110 Efficiency Discrepancy:  0.17985988544460818 +- 0.0010772168620260987
RPC Hits Efficiency Discrepancy:  0.9882278195694479 +- 6.547208291815

In [24]:
# total_DT_eff_dict = {"Data_lowMET": data_lowMET_counts_efficiencies, "MC_lowMET": MC_lowMET_counts_efficiencies,
#                      "Data_highMET": data_highMET_counts_efficiencies, "MC_highMET": MC_highMET_counts_efficiencies}
# dirName="Data_MC_Efficiencies"
# os.makedirs(dirName, exist_ok=True)
# with open(dirName+"/DT_Efficiencies_Dict.json", 'w') as f:
#         json.dump(total_DT_eff_dict,f)

In [25]:
#compute overall efficiencies for all campaigns combined
lowMET_selections = ["Forward_Eff", "clusterSize120_Eff","clusterSize160_Eff","RPC_Eff","RPCBX_Eff"]

data_combined_lowMET_counts_efficiencies = {}
for selection in lowMET_selections:
    #teff_list = []
    teff = rt.TEfficiency("teff", "teff", 1, 0, 1)
    for campaign, efficiencies_dict in data_lowMET_counts_efficiencies.items():
        #teff_list.append(efficiencies_dict[selection][3])
        teff += efficiencies_dict[selection][3]
    data_combined_lowMET_counts_efficiencies[selection] = teff.GetEfficiency(1),teff.GetEfficiencyErrorLow(1),teff.GetEfficiencyErrorUp(1)
    print(selection + " " + str(teff.GetEfficiency(1)))
MC_combined_lowMET_counts_efficiencies = {}
for selection in lowMET_selections:
    #teff_list = []
    teff = rt.TEfficiency("teff", "teff", 1, 0, 1)
    for campaign, efficiencies_dict in MC_lowMET_counts_efficiencies.items():
        #teff_list.append(efficiencies_dict[selection][3])
        teff += efficiencies_dict[selection][3]
    #teff = rt.TEfficiency.Merge(teff_list)
    MC_combined_lowMET_counts_efficiencies[selection] = teff.GetEfficiency(1),teff.GetEfficiencyErrorLow(1),teff.GetEfficiencyErrorUp(1)
    print(selection + " " + str(teff.GetEfficiency(1)))

Forward_Eff 0.005165443467414615
clusterSize120_Eff 0.012472073400863284
clusterSize160_Eff 0.0011062186842504826
RPC_Eff 0.9431056547296272
RPCBX_Eff 0.8425915883998872
Forward_Eff 0.0253809692278845
clusterSize120_Eff 0.04628800392359745
clusterSize160_Eff 0.013960498257562536
RPC_Eff 0.9982156492243613
RPCBX_Eff 0.9943785666570666


In [26]:
#compute uncertainties in data_combined/MC_combined discrepancy
print("EFFICIENCY DISCREPANCIES - LOW MET DTs")
print("##################################")
#print("# clusters pass forward veto (data_combined/MC_combined)", data_combined_lowMET_counts_efficiencies["Pass_Forward"], "/",MC_combined_lowMET_counts_efficiencies["Pass_Forward"])
#print("data_combined/MC_combined ratio clusters that pass forward veto", data_combined_lowMET_counts_efficiencies["Pass_Forward"]/MC_combined_lowMET_counts_efficiencies["Pass_Forward"])
print("Forward Veto Efficiency Discrepancy: ", data_combined_lowMET_counts_efficiencies["Forward_Eff"][0]/MC_combined_lowMET_counts_efficiencies["Forward_Eff"][0],"+-",findEffErr(data_combined_lowMET_counts_efficiencies["Forward_Eff"],MC_combined_lowMET_counts_efficiencies["Forward_Eff"]))
print("Cluster Size 120 Efficiency Discrepancy: ", data_combined_lowMET_counts_efficiencies["clusterSize120_Eff"][0]/MC_combined_lowMET_counts_efficiencies["clusterSize120_Eff"][0],"+-",findEffErr(data_combined_lowMET_counts_efficiencies["clusterSize120_Eff"],MC_combined_lowMET_counts_efficiencies["clusterSize120_Eff"]))
print("Cluster Size 160 Efficiency Discrepancy: ", data_combined_lowMET_counts_efficiencies["clusterSize160_Eff"][0]/MC_combined_lowMET_counts_efficiencies["clusterSize160_Eff"][0],"+-",findEffErr(data_combined_lowMET_counts_efficiencies["clusterSize160_Eff"],MC_combined_lowMET_counts_efficiencies["clusterSize160_Eff"]))
print("RPC Hits Efficiency Discrepancy: ", data_combined_lowMET_counts_efficiencies["RPC_Eff"][0]/MC_combined_lowMET_counts_efficiencies["RPC_Eff"][0],"+-",findEffErr(data_combined_lowMET_counts_efficiencies["RPC_Eff"],MC_combined_lowMET_counts_efficiencies["RPC_Eff"]))
print("RPCBX Efficiency Discrepancy: ", data_combined_lowMET_counts_efficiencies["RPCBX_Eff"][0]/MC_combined_lowMET_counts_efficiencies["RPCBX_Eff"][0],"+-",findEffErr(data_combined_lowMET_counts_efficiencies["RPCBX_Eff"],MC_combined_lowMET_counts_efficiencies["RPCBX_Eff"]))
#print("NStation10 Efficiency Discrepancy: ", data_combined_lowMET_counts_efficiencies["NStation10_Eff"][0]/MC_combined_lowMET_counts_efficiencies["NStation10_Eff"][0],"+-",findEffErr(data_combined_lowMET_counts_efficiencies["NStation10_Eff"],MC_combined_lowMET_counts_efficiencies["NStation10_Eff"]))

EFFICIENCY DISCREPANCIES - LOW MET DTs
##################################
Forward Veto Efficiency Discrepancy:  0.20351639927681178 +- 0.0006361744812974121
Cluster Size 120 Efficiency Discrepancy:  0.2694450471757126 +- 0.006821339180065354
Cluster Size 160 Efficiency Discrepancy:  0.07923919790264172 +- 0.00675280487670601
RPC Hits Efficiency Discrepancy:  0.9447914941650574 +- 0.0006310495774941073
RPCBX Efficiency Discrepancy:  0.8473549376999732 +- 0.0009939133180248774


In [27]:
highMET_selections = ["Forward_Eff", "clusterSize_Eff","RPC_Eff","RPCBX_Eff"]


data_combined_highMET_counts_efficiencies = {}
for selection in highMET_selections:
    #teff_list = []
    teff = rt.TEfficiency("teff", "teff", 1, 0, 1)
    for campaign, efficiencies_dict in data_highMET_counts_efficiencies.items():
        #teff_list.append(efficiencies_dict[selection][3])
        teff += efficiencies_dict[selection][3]
    data_combined_highMET_counts_efficiencies[selection] = teff.GetEfficiency(1),teff.GetEfficiencyErrorLow(1),teff.GetEfficiencyErrorUp(1)
    print(selection + " " + str(teff.GetEfficiency(1)))
MC_combined_highMET_counts_efficiencies = {}
for selection in highMET_selections:
    #teff_list = []
    teff = rt.TEfficiency("teff", "teff", 1, 0, 1)
    for campaign, efficiencies_dict in MC_highMET_counts_efficiencies.items():
        #teff_list.append(efficiencies_dict[selection][3])
        teff += efficiencies_dict[selection][3]
    #teff = rt.TEfficiency.Merge(teff_list)
    MC_combined_highMET_counts_efficiencies[selection] = teff.GetEfficiency(1),teff.GetEfficiencyErrorLow(1),teff.GetEfficiencyErrorUp(1)
    print(selection + " " + str(teff.GetEfficiency(1)))

Forward_Eff 0.9944875643971802
clusterSize_Eff 0.008213678436700154
RPC_Eff 0.9711824357368959
RPCBX_Eff 0.9073401777416888
Forward_Eff 0.9918655575765147
clusterSize_Eff 0.043120004314709415
RPC_Eff 0.9964955446211587
RPCBX_Eff 0.982507255809791


In [28]:
#compute uncertainties in data_combined/MC_combined discrepancy
print("EFFICIENCY DISCREPANCIES - HIGH MET DTs")
print("##################################")
#print("# clusters pass forward veto (data_combined/MC_combined)", data_combined_highMET_counts_efficiencies["Pass_Forward"], "/",MC_combined_highMET_counts_efficiencies["Pass_Forward"])
#print("data_combined/MC_combined ratio clusters that pass forward veto", data_combined_highMET_counts_efficiencies["Pass_Forward"]/MC_combined_highMET_counts_efficiencies["Pass_Forward"])
print("Forward Veto Efficiency Discrepancy: ", data_combined_highMET_counts_efficiencies["Forward_Eff"][0]/MC_combined_highMET_counts_efficiencies["Forward_Eff"][0],"+-",findEffErr(data_combined_highMET_counts_efficiencies["Forward_Eff"],MC_combined_highMET_counts_efficiencies["Forward_Eff"]))
print("Cluster Size 110 Efficiency Discrepancy: ", data_combined_highMET_counts_efficiencies["clusterSize_Eff"][0]/MC_combined_highMET_counts_efficiencies["clusterSize_Eff"][0],"+-",findEffErr(data_combined_highMET_counts_efficiencies["clusterSize_Eff"],MC_combined_highMET_counts_efficiencies["clusterSize_Eff"]))
#print("Cluster Size 160 Efficiency Discrepancy: ", data_combined_highMET_counts_efficiencies["clusterSize160_Eff"][0]/MC_combined_highMET_counts_efficiencies["clusterSize160_Eff"][0],"+-",findEffErr(data_combined_highMET_counts_efficiencies["clusterSize160_Eff"],MC_combined_highMET_counts_efficiencies["clusterSize160_Eff"]))
print("RPC Hits Efficiency Discrepancy: ", data_combined_highMET_counts_efficiencies["RPC_Eff"][0]/MC_combined_highMET_counts_efficiencies["RPC_Eff"][0],"+-",findEffErr(data_combined_highMET_counts_efficiencies["RPC_Eff"],MC_combined_highMET_counts_efficiencies["RPC_Eff"]))
print("RPCBX Efficiency Discrepancy: ", data_combined_highMET_counts_efficiencies["RPCBX_Eff"][0]/MC_combined_highMET_counts_efficiencies["RPCBX_Eff"][0],"+-",findEffErr(data_combined_highMET_counts_efficiencies["RPCBX_Eff"],MC_combined_highMET_counts_efficiencies["RPCBX_Eff"]))
#print("NStation10 Efficiency Discrepancy: ", data_combined_highMET_counts_efficiencies["NStation10_Eff"][0]/MC_combined_highMET_counts_efficiencies["NStation10_Eff"][0],"+-",findEffErr(data_combined_highMET_counts_efficiencies["NStation10_Eff"],MC_combined_highMET_counts_efficiencies["NStation10_Eff"]))

EFFICIENCY DISCREPANCIES - HIGH MET DTs
##################################
Forward Veto Efficiency Discrepancy:  1.0026435103030213 +- 2.7545905962247507e-05
Cluster Size 110 Efficiency Discrepancy:  0.19048417474063756 +- 0.00046649572846407946
RPC Hits Efficiency Discrepancy:  0.974597870486329 +- 3.573570447469239e-05
RPCBX Efficiency Discrepancy:  0.9234946331198859 +- 6.532340462609099e-05


In [29]:
#compute shift for MC to match data efficiency
new_MC_threshold_lowMET_120 = {}
for campaign in list(data_lowMET_counts_efficiencies.keys()):
    print(campaign)
    events_MC = events_MC_dict[campaign]
    data_eff = data_lowMET_counts_efficiencies[campaign]["clusterSize120_Eff"][0]
    #print(data_eff)
    MC_eff = MC_lowMET_counts_efficiencies[campaign]["clusterSize120_Eff"][0]
    MC_denom = ak.sum(ak.mask(events_MC.weight_total, makeEventMask(events_MC, [], ["forward_veto"], True)))
    if data_eff<MC_eff:
        for idx in range(120, 500):
            num_clusterSize = ak.sum(ak.mask(events_MC.weight_total, makeEventMask(events_MC, ['RPC_veto','RPCBX_veto'],["forward_veto"],clusterSize=idx)))
            #print(idx, num_clusterSize/MC_denom)
            if num_clusterSize/MC_denom < data_eff:
                new_MC_threshold_lowMET_120[campaign] = idx
                break
    if data_eff>MC_eff:
        for idx in range(120, 0, -1):
            num_clusterSize = ak.sum(ak.mask(events_MC.weight_total, makeEventMask(events_MC, ['RPC_veto','RPCBX_veto'],["forward_veto"],clusterSize=idx)))
            if num_clusterSize/MC_denom>data_eff:
                new_MC_threshold_lowMET_120[campaign] = idx
                break
print(new_MC_threshold_lowMET_120)

2022
2022EE
2023
2023BPix
2024
{'2022': 165, '2022EE': 163, '2023': 163, '2023BPix': 168, '2024': 164}


In [30]:
#compute shift for MC to match data efficiency - campaign combined
new_MC_threshold_lowMET_120 = {}

data_eff = data_combined_lowMET_counts_efficiencies["clusterSize120_Eff"][0]
#print(data_eff)
MC_eff = MC_combined_lowMET_counts_efficiencies["clusterSize120_Eff"][0]

MC_denom = 0
for campaign in list(MC_lowMET_counts_efficiencies.keys()):
    events_MC = events_MC_dict[campaign]
    MC_denom+=ak.sum(ak.mask(events_MC.weight_total, makeEventMask(events_MC, [], ["forward_veto"], True)))
    
if data_eff<MC_eff:
    for idx in range(120, 500):
        num_clusterSize=0
        for campaign in list(MC_lowMET_counts_efficiencies.keys()):
            events_MC = events_MC_dict[campaign]
            num_clusterSize += ak.sum(ak.mask(events_MC.weight_total, makeEventMask(events_MC, ['RPC_veto','RPCBX_veto'],["forward_veto"],clusterSize=idx)))
        #print(idx, num_clusterSize/MC_denom)
        if num_clusterSize/MC_denom < data_eff:
            new_MC_threshold_lowMET_120 = idx
            break
if data_eff>MC_eff:
    for idx in range(120, 0, -1):
        num_clusterSize=0
        for campaign in list(MC_lowMET_counts_efficiencies.keys()):
            events_MC = events_MC_dict[campaign]
            num_clusterSize += ak.sum(ak.mask(events_MC.weight_total, makeEventMask(events_MC, ['RPC_veto','RPCBX_veto'],["forward_veto"],clusterSize=idx)))
            new_MC_threshold_lowMET_120 = idx
            break
print(new_MC_threshold_lowMET_120)

164


In [31]:
#compute shift for MC to match data efficiency
new_MC_threshold_lowMET_160 = {}
for campaign in list(data_lowMET_counts_efficiencies.keys()):
    print(campaign)
    events_MC = events_MC_dict[campaign]
    data_eff = data_lowMET_counts_efficiencies[campaign]["clusterSize160_Eff"][0]
    #print(data_eff)
    MC_eff = MC_lowMET_counts_efficiencies[campaign]["clusterSize160_Eff"][0]
    MC_denom = ak.sum(ak.mask(events_MC.weight_total, makeEventMask(events_MC, [], ["forward_veto"], True)))
    if data_eff<MC_eff:
        for idx in range(160, 500):
            num_clusterSize = ak.sum(ak.mask(events_MC.weight_total, makeEventMask(events_MC, ['RPC_veto','RPCBX_veto'],["forward_veto"],clusterSize=idx)))
            #print(idx, num_clusterSize/MC_denom)
            if num_clusterSize/MC_denom < data_eff:
                new_MC_threshold_lowMET_160[campaign] = idx
                break
    if data_eff>MC_eff:
        for idx in range(160, 0, -1):
            num_clusterSize = ak.sum(ak.mask(events_MC.weight_total, makeEventMask(events_MC, ['RPC_veto','RPCBX_veto'],["forward_veto"],clusterSize=idx)))
            if num_clusterSize/MC_denom>data_eff:
                new_MC_threshold_lowMET_160[campaign] = idx
                break
print(new_MC_threshold_lowMET_160)

2022
2022EE
2023
2023BPix
2024
{'2022': 231, '2022EE': 233, '2023': 286, '2023BPix': 234, '2024': 235}


In [32]:
#compute shift for MC to match data efficiency - campaign combined
new_MC_threshold_lowMET_160 = {}
events_MC = events_MC_dict[campaign]
data_eff = data_combined_lowMET_counts_efficiencies["clusterSize160_Eff"][0]
#print(data_eff)
MC_eff = MC_combined_lowMET_counts_efficiencies["clusterSize160_Eff"][0]

MC_denom = 0
for campaign in list(MC_lowMET_counts_efficiencies.keys()):
    MC_denom+=ak.sum(ak.mask(events_MC.weight_total, makeEventMask(events_MC, [], ["forward_veto"], True)))
    
if data_eff<MC_eff:
    for idx in range(160, 500):
        num_clusterSize=0
        for campaign in list(MC_lowMET_counts_efficiencies.keys()):
            num_clusterSize += ak.sum(ak.mask(events_MC.weight_total, makeEventMask(events_MC, ['RPC_veto','RPCBX_veto'],["forward_veto"],clusterSize=idx)))
        #print(idx, num_clusterSize/MC_denom)
        if num_clusterSize/MC_denom < data_eff:
            new_MC_threshold_lowMET_160 = idx
            break
if data_eff>MC_eff:
    for idx in range(160, 0, -1):
        num_clusterSize=0
        for campaign in list(MC_lowMET_counts_efficiencies.keys()):
            num_clusterSize += ak.sum(ak.mask(events_MC.weight_total, makeEventMask(events_MC, ['RPC_veto','RPCBX_veto'],["forward_veto"],clusterSize=idx)))
            new_MC_threshold_lowMET_160 = idx
            break
print(new_MC_threshold_lowMET_160)

237


In [33]:
#compute shift for MC to match data efficiency
new_MC_threshold_highMET = {}
for campaign in list(data_highMET_counts_efficiencies.keys()):
    print(campaign)
    events_MC = events_MC_dict[campaign]
    data_eff = data_highMET_counts_efficiencies[campaign]["clusterSize_Eff"][0]
    #print(data_eff)
    MC_eff = MC_highMET_counts_efficiencies[campaign]["clusterSize_Eff"][0]
    MC_denom = ak.sum(ak.mask(events_MC.weight_total, makeEventMaskHighMET(events_MC, [], [], True)))
    if data_eff<MC_eff:
        for idx in range(110, 500):
            num_clusterSize = ak.sum(ak.mask(events_MC.weight_total, makeEventMaskHighMET(events_MC, ['RPC_veto','RPCBX_veto'],[],clusterSize=idx)))
            #print(idx, num_clusterSize/MC_denom)
            if num_clusterSize/MC_denom < data_eff:
                new_MC_threshold_highMET[campaign] = idx
                break
    if data_eff>MC_eff:
        for idx in range(110, 0, -1):
            num_clusterSize = ak.sum(ak.mask(events_MC.weight_total, makeEventMaskHighMET(events_MC, ['RPC_veto','RPCBX_veto'],[],clusterSize=idx)))
            if num_clusterSize/MC_denom>data_eff:
                new_MC_threshold_highMET[campaign] = idx
                break
print(new_MC_threshold_highMET)

2022
2022EE
2023
2023BPix
2024
{'2022': 160, '2022EE': 162, '2023': 160, '2023BPix': 162, '2024': 161}


In [34]:
#compute shift for MC to match data efficiency - campaign combined
new_MC_threshold_highMET_110 = {}
events_MC = events_MC_dict[campaign]
data_eff = data_combined_highMET_counts_efficiencies["clusterSize_Eff"][0]
#print(data_eff)
MC_eff = MC_combined_highMET_counts_efficiencies["clusterSize_Eff"][0]

MC_denom = 0
for campaign in list(MC_highMET_counts_efficiencies.keys()):
    MC_denom+=ak.sum(ak.mask(events_MC.weight_total, makeEventMask(events_MC, [], [], True)))
    
if data_eff<MC_eff:
    for idx in range(110, 500):
        num_clusterSize=0
        for campaign in list(MC_highMET_counts_efficiencies.keys()):
            num_clusterSize += ak.sum(ak.mask(events_MC.weight_total, makeEventMaskHighMET(events_MC, ['RPC_veto','RPCBX_veto'],[],clusterSize=idx)))
        #print(idx, num_clusterSize/MC_denom)
        if num_clusterSize/MC_denom < data_eff:
            new_MC_threshold_highMET_110 = idx
            break
if data_eff>MC_eff:
    for idx in range(110, 0, -1):
        num_clusterSize=0
        for campaign in list(MC_highMET_counts_efficiencies.keys()):
            num_clusterSize += ak.sum(ak.mask(events_MC.weight_total, makeEventMaskHighMET(events_MC, ['RPC_veto','RPCBX_veto'],[],clusterSize=idx)))
            new_MC_threshold_highMET_110 = idx
            break
print(new_MC_threshold_highMET_110)

161


In [35]:
DT_clusterSize_shift = {"LowMET_120": new_MC_threshold_lowMET_120, "LowMET_160": new_MC_threshold_lowMET_160, "HighMET": new_MC_threshold_highMET}
                     
dirName="Data_MC_Efficiencies"
os.makedirs(dirName, exist_ok=True)
with open(dirName+"/New_MC_thresholds_DTs.json", 'w') as f:
        json.dump(DT_clusterSize_shift,f)

### Helper functions to make histograms and style them appropriately

In [36]:
invertHotspot=False

In [37]:
clusterSize_bins = np.concatenate((np.linspace(0,130, 14), np.linspace(170, 570, 11)))
print(clusterSize_bins)
clusterSize_bins_highMET = np.concatenate((np.linspace(0,130, 14), np.linspace(170, 570, 11)))
print(clusterSize_bins_highMET)

[  0.  10.  20.  30.  40.  50.  60.  70.  80.  90. 100. 110. 120. 130.
 170. 210. 250. 290. 330. 370. 410. 450. 490. 530. 570.]
[  0.  10.  20.  30.  40.  50.  60.  70.  80.  90. 100. 110. 120. 130.
 170. 210. 250. 290. 330. 370. 410. 450. 490. 530. 570.]


In [38]:
rt.gStyle.SetOptStat(0)
def make_ratio_plot(campaign, h_list_in, title = "", label = "", fit = False, in_tags = None, ratio_bounds = [0.1, 4], logy = False, ratio_index = 0, draw_opt = ['E2','E1'], text = "", scale=False, scales = [1,1], return_dataMC = False):
    h_list = []
    if in_tags == None:
        tag = []
    else:
        tag = in_tags
    for i, h in enumerate(h_list_in):
        h_list.append(h.Clone('h{}aux{}'.format(i, label)))
        if in_tags == None:
            tag.append(h.GetTitle())
    #print("tags: ", tag)
    global c_out
    c_out = rt.TCanvas("c_out_ratio", "c_out_ratio", 800, 800)
    pad1 = rt.TPad("pad1a", "pad1a", 0, 0.3, 1, 1.0)
    pad1.SetBottomMargin(0.03)
    pad1.SetLeftMargin(0.15)
    pad1.SetRightMargin(0.04)# pad2.SetGrid()
    if logy:
        pad1.SetLogy()

    pad1.Draw()
    pad1.cd()

    leg = rt.TLegend(0.5, 0.65, 0.9, 0.92)
    leg = rt.TLegend(0.8, 0.75, 0.93, 0.9)

    #leg = rt.TLegend(0.2, 0.7, 0.5, 0.9)
    # leg = rt.TLegend(0.7, 0.2, 0.9, 0.4)
    leg.SetBorderSize(0)
    leg.SetTextSize(0.045)
    leg.SetFillStyle(0)
    c_out.cd(1)

    scaled_h_list = []
    if scale:
        for i, h_unscaled in enumerate(h_list):
            #h = h_unscaled.Clone()
            #h = h_unscaled.Scale(1/scales[i])
            #scaled_h_list.append(h_unscaled.Clone())
            h_unscaled.Scale(1/scales[i])
            scaled_h_list.append(h_unscaled)
    else:
        #for i, h_unscaled in enumerate(h_list):
            #h = h_unscaled.Clone()
            #scaled_h_list.append(h)
        scaled_h_list = h_list
    for i, h in enumerate(scaled_h_list):
        h.GetXaxis().SetLabelSize(0)
        h.GetXaxis().SetTitle(label)
        h.GetYaxis().SetRangeUser(0, 1.1*max(map(lambda x: x.GetMaximum(), scaled_h_list)))
        if logy and not scale:
            h.GetYaxis().SetRangeUser(10e-2, 2*max(map(lambda x: x.GetMaximum(), scaled_h_list)))
        if logy and scale:
            h.GetYaxis().SetRangeUser(10e-4, 1)
        h.GetYaxis().SetTitleOffset(1.0)
        h.GetYaxis().SetTitleSize(0.06)
        h.GetYaxis().SetLabelSize(0.05)
        
        if scale:
            y_title = "Fraction of Events"
        else:
            y_title = "Events"
        
        h.GetYaxis().SetTitle()
        h.SetTitle(f";{label};{y_title}")
        #if ratio_index == 0:h.DrawCopy("hist")
        '''
        h.SetFillColor(h_list_in[i].GetLineColor())
        h.SetFillStyle(3002)
        #h.SetStats(1)
        h.SetLineColor(h_list_in[i].GetLineColor())
        h.SetLineWidth(2)
        h.SetMarkerColor(h_list_in[i].GetLineColor())
        h.SetMarkerSize(2)
        # if ratio_index == 0:
        #     # h.DrawCopy("hist")
        #     h.DrawCopy(draw_opt[i]+'same')
        # else:h.DrawCopy(draw_opt[i])
        #if ratio_index == 0 :h.DrawCopy(draw_opt[i]+"same")
        #h.DrawCopy("E2 HIST")
        '''
        if i==0:
            h.SetLineWidth(4)
            h.DrawCopy("hist")
            #h.SetFillStyle(0)
            h.SetFillColor(h_list_in[i].GetLineColor())
            h.SetFillStyle(3002)
            #h.SetStats(1)
            h.SetLineColor(h_list_in[i].GetLineColor())
            h.SetLineWidth(2)
            h.SetMarkerColor(h_list_in[i].GetLineColor())
            h.SetMarkerSize(2)
            h.DrawCopy(draw_opt[i] + "same")
            #h.SetFillStyle(0)
        else:
            h.SetLineWidth(2)
            h.DrawCopy(draw_opt[i] + "same")
        #else:h.DrawCopy(draw_opt[i])
        if len(text)>0:
            l = rt.TLatex()
            l.SetTextSize(0.045)
            if logy:l.DrawLatex((h.GetXaxis().GetXmax()-h.GetXaxis().GetXmin())*0.1+h.GetXaxis().GetXmin() , h.GetMaximum()/10, text)
            else:l.DrawLatex((h.GetXaxis().GetXmax()-h.GetXaxis().GetXmin())*0.1+h.GetXaxis().GetXmin() , h.GetMaximum()*0.8, text)
        #if i==1:
            #h.DrawCopy(draw_opt[i]+"same")
       #     h.Draw("E1 same")
        if i==1:
            leg.AddEntry(h, tag[i], "lep")
        else:
            leg.AddEntry(h, tag[i], "l")
    leg.Draw("same")
    cmsText = rt.TLatex()

    cmsText.SetNDC(True)

    #labelStr = "CMS Preliminary " + campaignStr
    cmsText.SetTextFont(42)
    cmsText.SetTextSize(0.045)
    cmsText.SetTextAlign(11)
    if campaign=="2022":
        cmsText.DrawLatex(0.52, 0.91, "#bf{CMS Preliminary 2022} (7.98 fb^{-1})")
    if campaign=="2022EE":
        cmsText.DrawLatex(0.52, 0.91, "#bf{CMS Preliminary 2022} (26.67 fb^{-1})")
    if campaign=="2023":
        cmsText.DrawLatex(0.52, 0.91, "#bf{CMS Preliminary 2023} (18.41 fb^{-1})")
    if campaign=="2023BPix":
        cmsText.DrawLatex(0.52, 0.91, "#bf{CMS Preliminary 2023} (9.45 fb^{-1})")
    if campaign=="2024":
        cmsText.DrawLatex(0.52, 0.91, "#bf{CMS Preliminary 2024} (109.08 fb^{-1})")
    if campaign=="all":
        cmsText.DrawLatex(0.52, 0.91, "#bf{CMS Preliminary 2022, 2023, and 2024} (171.59 fb^{-1})")

    scaled_h_list_copy = [scaled_h_list[0].Clone(), scaled_h_list[1].Clone()]
    c_out.cd()
    pad2 = rt.TPad("pad2a", "pad2a", 0, 0, 1, 0.3)
    pad2.SetTopMargin(0.03)
    pad2.SetBottomMargin(0.25)
    pad2.SetLeftMargin(0.15)
    pad2.SetRightMargin(0.04)# pad2.SetGrid()
    pad2.Draw()
    pad2.cd()
    band = scaled_h_list[ratio_index].Clone('h_band')
    for j in range(band.GetXaxis().GetNbins()):
        band.SetBinContent(j+1, 1.0)
        if h_list[ratio_index].GetBinContent(j+1) == 0:
            band.SetBinError(j+1, 0.0)
        else:
            band.SetBinError(j+1, scaled_h_list[ratio_index].GetBinError(j+1)/scaled_h_list[ratio_index].GetBinContent(j+1))
            #print(j, h_list_in[0].GetBinError(j+1)/h_list_in[0].GetBinContent(j+1))
    band.SetFillColor(scaled_h_list[ratio_index].GetLineColor())

    band.SetFillStyle(3002)
    band.SetLineColor(scaled_h_list[ratio_index].GetLineColor())
    #band.SetFillColorAlpha(0,0)
    #band.SetLineColor(0)
    
    band.GetYaxis().SetTitleOffset(0.5)
    band.GetYaxis().SetRangeUser(ratio_bounds[0], ratio_bounds[1])
    band.GetYaxis().SetTitleSize(0.11)
    band.GetYaxis().SetLabelSize(0.12)
    band.GetYaxis().SetNdivisions(506)
    band.GetXaxis().SetTitleOffset(0.95)
    band.GetXaxis().SetTitleSize(0.12)
    band.GetXaxis().SetLabelSize(0.12)
    band.GetXaxis().SetTickSize(0.07)
    
    band.SetYTitle('Data / MC'.format(tag[ratio_index]))
    band.SetXTitle(label)
    band.SetTitle("")
    band.DrawCopy('E2')
    ln = rt.TLine(h.GetXaxis().GetXmin(), 1, h.GetXaxis().GetXmax(), 1)
    ln.SetLineWidth(3)
    ln.SetLineColor(scaled_h_list[ratio_index].GetLineColor())
    ln.DrawLine(h.GetXaxis().GetXmin(), 1, h.GetXaxis().GetXmax(), 1)
     
    #print(ratio_index)
    for i, h in enumerate(scaled_h_list):
        if i == ratio_index:
            continue
        else:
            if fit:h.GetFunction("expo")
            h.Divide(scaled_h_list[ratio_index])
            # h.GetYaxis().SetTitleOffset(0.6)
            h.GetYaxis().SetRangeUser(ratio_bounds[0], ratio_bounds[1])
            # h.GetYaxis().SetTitleSize(0.12)
            # h.GetYaxis().SetLabelSize(0.12)
            # h.GetYaxis().SetNdivisions(506)
            # h.GetXaxis().SetTitleOffset(0.95)
            # h.GetXaxis().SetTitleSize(0.12)
            # h.GetXaxis().SetLabelSize(0.12)
            # h.GetXaxis().SetTickSize(0.07)
            # h.SetYTitle('Ratio with {}'.format(tag[0]))
            # h.SetTitle("")
            #set relative error of ratio to be the relative error of data
            for j in range(h.GetXaxis().GetNbins()):
                if h_list[i].GetBinContent(j+1) == 0:
                    h.SetBinError(j+1, 0.0)
                else:
                    h.SetBinError(j+1, h_list_in[i].GetBinError(j+1)/h_list_in[i].GetBinContent(j+1)*h.GetBinContent(j+1))
            h.Draw('same'+draw_opt[i])
    
    pad2.Update()
    
    if return_dataMC:
        return h
    
    c_out.pad1 = pad1
    c_out.pad2 = pad2
    c_out.h_list = h_list
    c_out.leg = leg
    
    
    return c_out, scaled_h_list_copy, scaled_h_list[1]

In [39]:
#helper function to build histograms

def makeHists(events_data, events_MC, branch, mask_array, bins_tuple, highMET = False, invertHotspot=False):
    print(f"on branch {branch}")
    
    nbins, lowBin, highBin = bins_tuple
    
    #loop over three types of plots (no cuts, just forward veto, all cuts other than that measured)
    name_strs = ["noCuts", "forwardVeto", "allOtherCuts","normToDataCounts", "normToClusterSizeShape", "normToCountsAndShape"]
    masks = [[], [], mask_array, [],[],[]]
    masks_forwardVeto = [[],["forward_veto"],["forward_veto"],["forward_veto"],["forward_veto"],["forward_veto"]]
    masks_forwardVeto_highMET = [[],["forward_veto_highMET"], [],[],[],[]]
    mask_bools = [True, True, False,True,True,True,True]
    
    hist_info = {}
    for plotType, mask_list, mask_list_forwardVeto, mask_list_forwardVeto_highMET, mask_bool in zip(name_strs, masks, masks_forwardVeto, masks_forwardVeto_highMET, mask_bools):
        #compute relevant mask for particular plot
        if plotType=="modifiedForwardVeto" and highMET:continue
        if not highMET:
            mask_data = makeEventMask(events_data, mask_list, mask_list_forwardVeto, mask_bool, invertHotspot)
            mask_MC = makeEventMask(events_MC, mask_list, mask_list_forwardVeto, mask_bool, invertHotspot)
        else:
            mask_data = makeEventMaskHighMET(events_data, mask_list, mask_list_forwardVeto_highMET, mask_bool, invertHotspot)
            mask_MC = makeEventMaskHighMET(events_MC, mask_list, mask_list_forwardVeto_highMET, mask_bool, invertHotspot)
    
        data_tree = events_data[mask_data]
        data_tree = data_tree[~ak.is_none(data_tree)]
        
        MC_tree = events_MC[mask_MC]
        MC_tree = MC_tree[~ak.is_none(MC_tree)]
    
        #initialize data and MC histograms
        if branch=="dtRechitClusterSize" and highMET:
            data = rt.TH1F("Data", "Data", nbinsx=np.size(clusterSize_bins_highMET)-1, xbins=clusterSize_bins_highMET)
            MC = rt.TH1F("MC", "MC", nbinsx=np.size(clusterSize_bins_highMET)-1, xbins=clusterSize_bins_highMET)
        elif branch=="dtRechitClusterSize" and not highMET:
            data = rt.TH1F("Data", "Data", nbinsx=np.size(clusterSize_bins)-1, xbins=clusterSize_bins)
            MC = rt.TH1F("MC", "MC", nbinsx=np.size(clusterSize_bins)-1, xbins=clusterSize_bins)
        else:
            data = rt.TH1F("Data", "Data", nbinsx=nbins, xlow = lowBin, xup=highBin)
            MC = rt.TH1F("MC", "MC", nbinsx=nbins, xlow = lowBin, xup=highBin)


        #build data hist
        data_arr = np.array(data_tree[branch], dtype=np.float64)
        data_size = np.size(data_arr)
        data_weights = np.ones(data_size, dtype=np.float64)
        data.FillN(data_size, data_arr, data_weights)
        data.SetLineColor(rt.kBlack)
        data.SetFillStyle(0)
        
        #build MC hist
        MC_arr = np.array(MC_tree[branch], dtype=np.float64)
        MC_size = np.size(MC_arr)
        MC_weights = np.array(MC_tree["weight_total"])
        if plotType=="normToDataCounts":
            if not highMET:MC_weights = MC_weights*np.array(MC_tree["overallNormalizationToData"])
            else:MC_weights = MC_weights*np.array(MC_tree["overallNormalizationToData_highMET"])
#         if plotType=="normToClusterSizeShape":
#             if not highMET:MC_weights = MC_weights*np.array(MC_tree["clusterSizeScaleFactor"])
#             else:MC_weights = MC_weights*np.array(MC_tree["clusterSizeScaleFactor_highMET"])
#         if plotType=="normToCountsAndShape":
#             if not highMET:MC_weights = MC_weights*np.array(MC_tree["clusterSizeScaleFactor"])*np.array(MC_tree["overallNormalizationToData"])
#             else:MC_weights = MC_weights*np.array(MC_tree["clusterSizeScaleFactor_highMET"])*np.array(MC_tree["overallNormalizationToData_highMET"])
        MC.FillN(MC_size, MC_arr, MC_weights)
        MC.SetLineColor(rt.kRed)
        MC.SetFillStyle(0)
        
        sumOfWeights = np.sum(MC_weights)
        
        hist_info[branch+"_"+plotType] = {"MC_hist": MC, "data_hist": data, 
                                          "MC_weights": sumOfWeights, "data_weights": data_size}
        
    return hist_info
    

In [40]:
#define dictionary with relevant plot info
plot_info = {
             "ZMass": {"filename_base":"ZMass", "title":"Dimuon Mass Distribution", 
                        "xlabel":"Dimuon Mass [GeV]", "masks":[],"bins":(80, 0, 150), "logy":False},
#              "puppiMet": {"filename_base":"puppiMet", "title":"PUPPI MET Distribution", 
#                         "xlabel":"PUPPI MET [GeV]", "masks":[],"bins":(60, -5, 100), "logy":False},
#              "puppiMetPhi": {"filename_base":"puppiMetPhi", "title":"PUPPI MET Phi Distribution", 
#                         "xlabel":"PUPPI MET Phi", "masks":[],"bins":(30, -4, 4), "logy":False},
#              "dtRechitClusterMuonDeltaR": {"filename_base":"cluster_muon_deltaR", "title":"deltaR(cluster, muon)", 
#                         "xlabel":"deltaR(cluster, muon)", "masks":[],"bins":(50, 0, 0.5), "logy":False},
            
              "probe_pT": {"filename_base":"probe_pT", "title":"pT of Muon Matched to Cluster", 
                         "xlabel":"pT [GeV]", "masks":[],"bins":(25, 0, 100), "logy":False},
#             "tag_pT": {"filename_base":"tag_pT", "title":"pT of Muon Not Matched to Cluster", 
#                        "xlabel":"pT [GeV]", "masks":[],"bins":(25, 0, 100), "logy":False},
#             "probe_phi": {"filename_base":"probe_phi", "title":"Phi of Muon Matched to Cluster", 
#                        "xlabel":"pT [GeV]", "masks":[],"bins":(60, -4, 4), "logy":False},
#             "tag_phi": {"filename_base":"tag_phi", "title":"Phi of Muon Not Matched to Cluster", 
#                        "xlabel":"pT [GeV]", "masks":[],"bins":(60, -4, 4), "logy":False},
#             "probe_eta": {"filename_base":"probe_eta", "title":"Eta of Muon Matched to Cluster", 
#                        "xlabel":"pT [GeV]", "masks":[],"bins":(60, -4, 4), "logy":False},
#             "tag_eta": {"filename_base":"tag_eta", "title":"Eta of Muon Not Matched to Cluster", 
#                        "xlabel":"pT [GeV]", "masks":[],"bins":(60, -4, 4), "logy":False},
            
            "dtRechitClusterSize": {"filename_base":"dtRechitClusterSize", "title":"Cluster Size Distribution", 
                       "xlabel":"N_{hits}", "masks":["clusterSize_veto"],"bins":(40, 0, 200), "logy":True},
            "dtRechitCluster_match_RPChits_dPhi0p5": {"filename_base":"MatchedRPCHits", "title":"Number of Matched RPC Hits", 
                       "xlabel":"Matched RPC Hits", "masks":["RPC_veto"],"bins":(26, -1, 26), "logy":False},
            "dtRechitCluster_match_RPCBx_dPhi0p5": {"filename_base":"RPCBX", "title":"Bunch Crossing with Most Hits", 
                       "xlabel":"Bunch Crossing", "masks":["RPCBX_veto"],"bins":(10, -4, 6), "logy":False}
#             "dtRechitClusterEta": {"filename_base":"cluster_eta", "title":"Cluster Eta", 
#                        "xlabel":"Eta", "masks":[],"bins":(60, -2.5, 2.5), "logy":False},
#             "dtRechitClusterPhi": {"filename_base":"cluster_phi", "title":"Cluster Phi", 
#                       "xlabel":"Phi", "masks":[],"bins":(60, -4, 4), "logy":False}
#             "cscRechitClusterXSpread": {"filename_base":"cscRechitClusterXSpread", "title":"Cluster X Spread", 
#                        "xlabel":"X Spread [cm]", "masks":[],"bins":(25, -5, 150), "logy": False},
#             "cscRechitClusterYSpread": {"filename_base":"cscRechitClusterYSpread", "title":"Cluster Y Spread", 
#                        "xlabel":"Y Spread [cm]", "masks":[],"bins":(25, -5, 150), "logy": False},
#             "cscRechitClusterZSpread": {"filename_base":"cscRechitClusterZSpread", "title":"Cluster Z Spread", 
#                        "xlabel":"Z Spread [cm]", "masks":[],"bins":(25, -5, 200), "logy": False},
#             "cscRechitClusterXYSpread": {"filename_base":"cscRechitClusterXYSpread", "title":"Cluster XY Spread", 
#                        "xlabel":"XY Spread [cm]", "masks":[],"bins":(25, -5, 150), "logy": False},
#             "cscRechitClusterRSpread": {"filename_base":"cscRechitClusterRSpread", "title":"Cluster R Spread", 
#                        "xlabel":"R Spread [cm]", "masks":[],"bins":(25, -5, 150), "logy": False},
#             "cscRechitClusterSkewX": {"filename_base":"cscRechitClusterSkewX", "title":"Cluster X Skew", 
#                        "xlabel":"X Skew [cm]", "masks":[],"bins":(25, -150, 150), "logy": False},
#             "cscRechitClusterSkewY": {"filename_base":"cscRechitClusterSkewY", "title":"Cluster Y Skew", 
#                        "xlabel":"Y Skew [cm]", "masks":[],"bins":(25, -150, 150), "logy": False},
#              "cscRechitClusterSkewZ": {"filename_base":"cscRechitClusterSkewZ", "title":"Cluster Z Skew", 
#                        "xlabel":"Z Skew [cm]", "masks":[],"bins":(25, -150, 150), "logy": False},
#             "cscRechitClusterSkewX": {"filename_base":"cscRechitClusterSkewX", "title":"Cluster X Spread", 
#                        "xlabel":"X Skew [cm]", "masks":[],"bins":(25, -150, 150), "logy": False},
#             "cscRechitClusterFracS1": {"filename_base":"cscRechitClusterFracS1", "title":"Fraction of Hits in Station 1", 
#                        "xlabel":"Station 1 Hits/Total Hits", "masks":[],"bins":(25, 0, 1.1), "logy": False},
#             "cscRechitClusterFracS2": {"filename_base":"cscRechitClusterFracS2", "title":"Fraction of Hits in Station 2", 
#                        "xlabel":"Station 2 Hits/Total Hits", "masks":[],"bins":(25, 0, 1.1), "logy": False},
#             "cscRechitClusterFracS3": {"filename_base":"cscRechitClusterFracS3", "title":"Fraction of Hits in Station 3", 
#                        "xlabel":"Station 3 Hits/Total Hits", "masks":[],"bins":(25, 0, 1.1), "logy": False},
#             "cscRechitClusterFracS4": {"filename_base":"cscRechitClusterFracS4", "title":"Fraction of Hits in Station 4", 
#                        "xlabel":"Station 4 Hits/Total Hits", "masks":[],"bins":(25, 0, 1.1), "logy": False},
#             "cscRechitClusterFracR1": {"filename_base":"cscRechitClusterFracR1", "title":"Fraction of Hits in Ring 1", 
#                        "xlabel":"Ring 1 Hits/Total Hits", "masks":[],"bins":(25, 0, 1.1), "logy": False},
#             "cscRechitClusterFracR2": {"filename_base":"cscRechitClusterFracR2", "title":"Fraction of Hits in Ring 2", 
#                        "xlabel":"Ring 2 Hits/Total Hits", "masks":[],"bins":(25, 0, 1.1), "logy": False},
#             "cscRechitClusterFracR3": {"filename_base":"cscRechitClusterFracR3", "title":"Fraction of Hits in Ring 3", 
#                        "xlabel":"Ring 3 Hits/Total Hits", "masks":[],"bins":(25, 0, 1.1), "logy": False}
            
            }

In [41]:
full_individual_plot_info = {}
for campaign in list(events_MC_dict.keys()):
    print(campaign)
    events_MC = events_MC_dict[campaign]
    events_data = events_data_dict[campaign]
    individual_plot_info = {}
    for branch, info_dict in plot_info.items():
        #if branch!="cscRechitClusterSize" and branch!="cscRechitClusterDNN_bkgMC_plusBeamHalo":continue
        filename_base = info_dict["filename_base"]
        hist_dict = makeHists(events_data, events_MC, branch, info_dict["masks"], info_dict["bins"])
        for plot_hists, plot_hist_dict in hist_dict.items():
            individual_plot_info[plot_hists] = {"MC_hist": plot_hist_dict["MC_hist"], "data_hist": plot_hist_dict["data_hist"], 
                        "file_name": plot_hists, "title": info_dict["title"], "label": info_dict["xlabel"], 
                        "scales": [plot_hist_dict["MC_weights"], plot_hist_dict["data_weights"]], "logy": info_dict["logy"]}
    full_individual_plot_info[campaign] = individual_plot_info

2022
on branch ZMass
on branch probe_pT
on branch dtRechitClusterSize
on branch dtRechitCluster_match_RPChits_dPhi0p5
on branch dtRechitCluster_match_RPCBx_dPhi0p5
2022EE
on branch ZMass
on branch probe_pT
on branch dtRechitClusterSize
on branch dtRechitCluster_match_RPChits_dPhi0p5
on branch dtRechitCluster_match_RPCBx_dPhi0p5
2023
on branch ZMass
on branch probe_pT
on branch dtRechitClusterSize
on branch dtRechitCluster_match_RPChits_dPhi0p5
on branch dtRechitCluster_match_RPCBx_dPhi0p5
2023BPix
on branch ZMass
on branch probe_pT
on branch dtRechitClusterSize
on branch dtRechitCluster_match_RPChits_dPhi0p5
on branch dtRechitCluster_match_RPCBx_dPhi0p5
2024
on branch ZMass
on branch probe_pT
on branch dtRechitClusterSize
on branch dtRechitCluster_match_RPChits_dPhi0p5
on branch dtRechitCluster_match_RPCBx_dPhi0p5


Warning in <TROOT::Append>: Replacing existing TH1: Data (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: MC (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Data (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: MC (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Data (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: MC (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Data (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: MC (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Data (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: MC (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Data (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: MC (Potential memory leak).
Warning in <TROOT::Append>: 

In [42]:
full_individual_plot_info_highMET = {}
for campaign in list(events_MC_dict.keys()):
    print(campaign)
    events_MC = events_MC_dict[campaign]
    events_data = events_data_dict[campaign]
    individual_plot_info_highMET = {}
    for branch, info_dict in plot_info.items():
        filename_base = info_dict["filename_base"]
        hist_dict = makeHists(events_data, events_MC, branch, info_dict["masks"], info_dict["bins"], True)
        for plot_hists, plot_hist_dict in hist_dict.items():
            individual_plot_info_highMET[plot_hists] = {"MC_hist": plot_hist_dict["MC_hist"], "data_hist": plot_hist_dict["data_hist"], 
                        "file_name": plot_hists, "title": info_dict["title"], "label": info_dict["xlabel"], 
                        "scales": [plot_hist_dict["MC_weights"], plot_hist_dict["data_weights"]], "logy": info_dict["logy"]}
    full_individual_plot_info_highMET[campaign] = individual_plot_info_highMET

2022
on branch ZMass
on branch probe_pT
on branch dtRechitClusterSize
on branch dtRechitCluster_match_RPChits_dPhi0p5
on branch dtRechitCluster_match_RPCBx_dPhi0p5
2022EE
on branch ZMass
on branch probe_pT
on branch dtRechitClusterSize
on branch dtRechitCluster_match_RPChits_dPhi0p5
on branch dtRechitCluster_match_RPCBx_dPhi0p5
2023
on branch ZMass
on branch probe_pT
on branch dtRechitClusterSize
on branch dtRechitCluster_match_RPChits_dPhi0p5
on branch dtRechitCluster_match_RPCBx_dPhi0p5
2023BPix
on branch ZMass
on branch probe_pT
on branch dtRechitClusterSize
on branch dtRechitCluster_match_RPChits_dPhi0p5
on branch dtRechitCluster_match_RPCBx_dPhi0p5
2024
on branch ZMass
on branch probe_pT
on branch dtRechitClusterSize
on branch dtRechitCluster_match_RPChits_dPhi0p5
on branch dtRechitCluster_match_RPCBx_dPhi0p5


Warning in <TROOT::Append>: Replacing existing TH1: Data (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: MC (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Data (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: MC (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Data (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: MC (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Data (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: MC (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Data (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: MC (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: Data (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: MC (Potential memory leak).
Warning in <TROOT::Append>: 

In [43]:
for campaign, individual_plot_info in full_individual_plot_info.items():
    print(campaign)
    plot_output = f"{campaign}_DT_Data_MC_Comp_finalzedSelections_DTs"
    os.makedirs(plot_output, exist_ok=True)

    for plot_type, plot_info_dict in individual_plot_info.items():
        print(plot_type)
        for boolScale in [True, False]:
            c, h_list, ratio = make_ratio_plot(campaign, [plot_info_dict["MC_hist"], plot_info_dict["data_hist"]], title = plot_info_dict["title"], label = plot_info_dict["label"], fit = False, in_tags = None, ratio_bounds = [0.1, 4], logy = plot_info_dict["logy"], ratio_index = 0, draw_opt = ['E2','E1'], text = "", scale=boolScale, scales = plot_info_dict["scales"])
            if boolScale:
                scaleString = "_normalized"
            else:
                scaleString=""
            os.makedirs(plot_output+"/"+plot_info_dict["file_name"], exist_ok=True)
            c.SaveAs(plot_output+"/"+plot_info_dict["file_name"]+"/"+plot_info_dict["file_name"]+scaleString+".png")
            file = rt.TFile(plot_output+"/"+plot_info_dict["file_name"]+"/"+plot_info_dict["file_name"]+scaleString+".root", "RECREATE")
            #file.cd()
            c.Write()
            h_list[0].Write("MC_hist")
            h_list[1].Write("Data_hist")
            ratio.Write("Data_MC_Ratio")
            file.Close()

2022
ZMass_noCuts
ZMass_forwardVeto
ZMass_allOtherCuts
ZMass_normToDataCounts
ZMass_normToClusterSizeShape
ZMass_normToCountsAndShape
probe_pT_noCuts
probe_pT_forwardVeto
probe_pT_allOtherCuts
probe_pT_normToDataCounts
probe_pT_normToClusterSizeShape
probe_pT_normToCountsAndShape
dtRechitClusterSize_noCuts
dtRechitClusterSize_forwardVeto
dtRechitClusterSize_allOtherCuts
dtRechitClusterSize_normToDataCounts
dtRechitClusterSize_normToClusterSizeShape
dtRechitClusterSize_normToCountsAndShape
dtRechitCluster_match_RPChits_dPhi0p5_noCuts
dtRechitCluster_match_RPChits_dPhi0p5_forwardVeto
dtRechitCluster_match_RPChits_dPhi0p5_allOtherCuts
dtRechitCluster_match_RPChits_dPhi0p5_normToDataCounts
dtRechitCluster_match_RPChits_dPhi0p5_normToClusterSizeShape
dtRechitCluster_match_RPChits_dPhi0p5_normToCountsAndShape
dtRechitCluster_match_RPCBx_dPhi0p5_noCuts
dtRechitCluster_match_RPCBx_dPhi0p5_forwardVeto
dtRechitCluster_match_RPCBx_dPhi0p5_allOtherCuts
dtRechitCluster_match_RPCBx_dPhi0p5_normToDat

Info in <TCanvas::Print>: png file 2022_DT_Data_MC_Comp_finalzedSelections_DTs/ZMass_noCuts/ZMass_noCuts_normalized.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_out_ratio
Info in <TCanvas::Print>: png file 2022_DT_Data_MC_Comp_finalzedSelections_DTs/ZMass_noCuts/ZMass_noCuts.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_out_ratio
Info in <TCanvas::Print>: png file 2022_DT_Data_MC_Comp_finalzedSelections_DTs/ZMass_forwardVeto/ZMass_forwardVeto_normalized.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_out_ratio
Info in <TCanvas::Print>: png file 2022_DT_Data_MC_Comp_finalzedSelections_DTs/ZMass_forwardVeto/ZMass_forwardVeto.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_out_ratio
Info in <TCanvas::Print>: png file 2022_DT_Data_MC_Comp_finalzedSelections_DTs/ZMass_allOtherCuts/ZMass_allOtherCuts_normalized.png has be

In [44]:
for campaign, individual_plot_info_highMET in full_individual_plot_info_highMET.items():
    print(campaign)

    plot_output = f"{campaign}_DT_Data_MC_Comp_finalzedSelections_highMET_DTs"
    os.makedirs(plot_output, exist_ok=True)

    for plot_type, plot_info_dict in individual_plot_info_highMET.items():
        print(plot_type)
        for boolScale in [True, False]:
            c, h_list, ratio = make_ratio_plot(campaign, [plot_info_dict["MC_hist"], plot_info_dict["data_hist"]], title = plot_info_dict["title"], label = plot_info_dict["label"], fit = False, in_tags = None, ratio_bounds = [0.1, 4], logy = plot_info_dict["logy"], ratio_index = 0, draw_opt = ['E2','E1'], text = "", scale=boolScale, scales = plot_info_dict["scales"])
            if boolScale:
                scaleString = "_normalized"
            else:
                scaleString=""
            os.makedirs(plot_output+"/"+plot_info_dict["file_name"], exist_ok=True)
            c.SaveAs(plot_output+"/"+plot_info_dict["file_name"]+"/"+plot_info_dict["file_name"]+scaleString+".png")
            file = rt.TFile(plot_output+"/"+plot_info_dict["file_name"]+"/"+plot_info_dict["file_name"]+scaleString+".root", "RECREATE")
            #file.cd()
            c.Write()
            h_list[0].Write("MC_hist")
            h_list[1].Write("Data_hist")
            ratio.Write("Data_MC_Ratio")
            file.Close()

2022
ZMass_noCuts
ZMass_forwardVeto
ZMass_allOtherCuts
ZMass_normToDataCounts
ZMass_normToClusterSizeShape
ZMass_normToCountsAndShape
probe_pT_noCuts
probe_pT_forwardVeto
probe_pT_allOtherCuts
probe_pT_normToDataCounts
probe_pT_normToClusterSizeShape
probe_pT_normToCountsAndShape
dtRechitClusterSize_noCuts
dtRechitClusterSize_forwardVeto
dtRechitClusterSize_allOtherCuts
dtRechitClusterSize_normToDataCounts
dtRechitClusterSize_normToClusterSizeShape
dtRechitClusterSize_normToCountsAndShape
dtRechitCluster_match_RPChits_dPhi0p5_noCuts
dtRechitCluster_match_RPChits_dPhi0p5_forwardVeto
dtRechitCluster_match_RPChits_dPhi0p5_allOtherCuts
dtRechitCluster_match_RPChits_dPhi0p5_normToDataCounts
dtRechitCluster_match_RPChits_dPhi0p5_normToClusterSizeShape
dtRechitCluster_match_RPChits_dPhi0p5_normToCountsAndShape
dtRechitCluster_match_RPCBx_dPhi0p5_noCuts
dtRechitCluster_match_RPCBx_dPhi0p5_forwardVeto
dtRechitCluster_match_RPCBx_dPhi0p5_allOtherCuts
dtRechitCluster_match_RPCBx_dPhi0p5_normToDat

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_out_ratio
Info in <TCanvas::Print>: png file 2022_DT_Data_MC_Comp_finalzedSelections_highMET_DTs/ZMass_noCuts/ZMass_noCuts_normalized.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_out_ratio
Info in <TCanvas::Print>: png file 2022_DT_Data_MC_Comp_finalzedSelections_highMET_DTs/ZMass_noCuts/ZMass_noCuts.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_out_ratio
Info in <TCanvas::Print>: png file 2022_DT_Data_MC_Comp_finalzedSelections_highMET_DTs/ZMass_forwardVeto/ZMass_forwardVeto_normalized.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_out_ratio
Info in <TCanvas::Print>: png file 2022_DT_Data_MC_Comp_finalzedSelections_highMET_DTs/ZMass_forwardVeto/ZMass_forwardVeto.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_out_ratio
Info in <TCanvas::Print>: p

In [45]:
import json

dirName="Data_MC_ClusterSize_JSONs_DTs"
os.makedirs(dirName, exist_ok=True)
for campaign, individual_plot_info in full_individual_plot_info.items():
    os.makedirs(dirName+"/"+campaign, exist_ok=True)
    MC_hist = individual_plot_info["dtRechitClusterSize_forwardVeto"]["MC_hist"].Clone()
    data_hist = individual_plot_info["dtRechitClusterSize_forwardVeto"]["data_hist"].Clone()
    h_list = [MC_hist, data_hist]
    scales = individual_plot_info["dtRechitClusterSize_forwardVeto"]["scales"]
#     scaled_h_list = []
#     for i, h_unscaled in enumerate(h_list):
#         #h = h_unscaled.Clone()
#         #h = h_unscaled.Scale(1/scales[i])
#         #scaled_h_list.append(h_unscaled.Clone())
#         h_unscaled.Scale(1/scales[i])
#         scaled_h_list.append(h_unscaled)
    
#     data_hist = scaled_h_list[1]
#     MC_hist = scaled_h_list[0]
    
#     #data_hist.Divide(MC_hist)
#     JSON_ShapeDifference = {}
#     JSON_BinEdges = {}
#     for j in range(MC_hist.GetXaxis().GetNbins()):
#         MC_val = MC_hist.GetBinContent(j)
#         if MC_val==0:continue
#         data_val = data_hist.GetBinContent(j)
#         JSON_BinEdges[j] = [MC_hist.GetBinLowEdge(j), MC_hist.GetBinLowEdge(j)+MC_hist.GetBinWidth(j)]
#         JSON_ShapeDifference[j] = data_val/MC_val
    
#     with open(dirName+"/"+campaign+"/clusterSize_shape_scaleFactors.json", "w") as f:
#         json.dump(JSON_ShapeDifference, f)
    
#     with open(dirName+"/"+campaign+"/clusterSize_bins.json", "w") as f:
#         json.dump(JSON_BinEdges, f)
        
    
#     #MC_eventCounts = MC_hist.GetEntries() DOESN'T INCORPORATE HISTOGRAM WEIGHTS
    data_eventCounts = data_hist.GetEntries()
    with open(dirName+"/"+campaign+"/forwardVeto_eventCounts.json", "w") as f:
        json.dump({"data_counts": scales[1], "MC_counts": scales[0]}, f)

In [46]:
import json

dirName="Data_MC_ClusterSize_JSONs_highMET_DTs"
os.makedirs(dirName, exist_ok=True)
for campaign, individual_plot_info in full_individual_plot_info_highMET.items():
    os.makedirs(dirName+"/"+campaign, exist_ok=True)
    MC_hist = individual_plot_info["dtRechitClusterSize_noCuts"]["MC_hist"].Clone()
    data_hist = individual_plot_info["dtRechitClusterSize_noCuts"]["data_hist"].Clone()
    h_list = [MC_hist, data_hist]
    scales = individual_plot_info["dtRechitClusterSize_noCuts"]["scales"]
#     scaled_h_list = []
#     for i, h_unscaled in enumerate(h_list):
#         #h = h_unscaled.Clone()
#         #h = h_unscaled.Scale(1/scales[i])
#         #scaled_h_list.append(h_unscaled.Clone())
#         h_unscaled.Scale(1/scales[i])
#         scaled_h_list.append(h_unscaled)
    
#     data_hist = scaled_h_list[1]
#     MC_hist = scaled_h_list[0]
    
#     #data_hist.Divide(MC_hist)
#     JSON_ShapeDifference = {}
#     JSON_BinEdges = {}
#     for j in range(MC_hist.GetXaxis().GetNbins()):
#         MC_val = MC_hist.GetBinContent(j)
#         if MC_val==0:continue
#         data_val = data_hist.GetBinContent(j)
#         JSON_BinEdges[j] = [MC_hist.GetBinLowEdge(j), MC_hist.GetBinLowEdge(j)+MC_hist.GetBinWidth(j)]
#         JSON_ShapeDifference[j] = data_val/MC_val
    
#     with open(dirName+"/"+campaign+"/clusterSize_shape_scaleFactors.json", "w") as f:
#         json.dump(JSON_ShapeDifference, f)
    
#     with open(dirName+"/"+campaign+"/clusterSize_bins.json", "w") as f:
#         json.dump(JSON_BinEdges, f)
        
    
#     #MC_eventCounts = MC_hist.GetEntries() DOESN'T INCORPORATE HISTOGRAM WEIGHTS
    data_eventCounts = data_hist.GetEntries()
    with open(dirName+"/"+campaign+"/forwardVeto_eventCounts.json", "w") as f:
        json.dump({"data_counts": scales[1], "MC_counts": scales[0]}, f)